# Description

Experiments conducted for the centralized data scenario. In this way, we seek to observe the behavior of entropy, its performance, and relationships.

Author: Ernesto Gurgel Valente Neto


# MNIST
* A dataset of handwritten digits, containing 70,000 grayscale images of 28x28 pixels, often used for training in image recognition techniques and machine learning.


## Creation and Definition of All Main Functions

1. Entropy Filter
2. Loading, Preprocessing, and Handling
3. Model Creation
4. Statistical Predictions
5. Accuracy and Loss Plot


In [ ]:
from keras.datasets import mnist
import numpy as np
from scipy.stats import entropy
import matplotlib.pyplot as plt
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Sequential
from keras.optimizers import SGD
from sklearn import metrics
from sklearn.metrics import accuracy_score, recall_score
from keras.utils import to_categorical
from keras.layers import Dropout
from keras.layers import BatchNormalization
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import time
from sklearn.model_selection import train_test_split

def entropia(pk, base=2, max=1):
    pk = pk / np.sum(pk)
    pk = pk[pk > 0]
    result = -np.sum(pk * np.log(pk) / np.log(base))
    return result

def filtrar_entropia_classe(train_X, train_y, totalClasses):  # filtro de entropia
    train_Xextend = []
    train_yextend = []
    for label in range(totalClasses):
        indices_originais = np.where(train_y == label)[0] #pega os indices originais da classe em label range
        indicesDasImagensDaClasse = train_X[indices_originais] #Os indices das imagens da classe são armazenados
        #Calculo a entropia para cada imagem da classe e em seguida enumera os indices da classe.
        tuplas = [(indices_originais[index], entropia(img)) for index, img in enumerate(indicesDasImagensDaClasse)]
        #Ordena as tuplas com base no segundo elemento, ou seja com base na entropia
        local_ordenado = sorted(tuplas, key=lambda x: x[1])
        #pega o tamanho da classe ordenada e separa a mediana da classe.
        n = len(local_ordenado)
        if n % 2 == 1:
            median = local_ordenado[n // 2][1]
        else:
            median = (local_ordenado[n // 2 - 1][1] + local_ordenado[n // 2][1]) / 2.0
        #Todas as imagens ordenadas pela entropia abaixo da medina são armezenadas nos indices_filtrados_da_Classe
        indices_filtrados_da_classe = [item[0] for item in local_ordenado if item[1] <= median]
        #as imagens abaixo da mediana são anexadas a cada nova interação em uma nova variavel.
        train_Xextend.extend([train_X[i] for i in indices_filtrados_da_classe])
        train_yextend.extend([train_y[i] for i in indices_filtrados_da_classe])
    return np.array(train_Xextend), np.array(train_yextend)

def filtrar_entropia_median_train_teste(train_X, train_y, test_X, test_y, totalClasses):  # filtro de entropia
    train_Xextend = []
    train_yextend = []
    for label in range(totalClasses):
        indices_originais = np.where(train_y == label)[0] #pega os indices originais da classe em label range
        indicesDasImagensDaClasse = train_X[indices_originais] #Os indices das imagens da classe são armazenados
        #Calculo a entropia para cada imagem da classe e em seguida enumera os indices da classe.
        tuplas = [(indices_originais[index], entropia(img)) for index, img in enumerate(indicesDasImagensDaClasse)]
        #Ordena as tuplas com base no segundo elemento, ou seja com base na entropia
        local_ordenado = sorted(tuplas, key=lambda x: x[1])
        #pega o tamanho da classe ordenada e separa a mediana da classe.
        n = len(local_ordenado)
        if n % 2 == 1:
            median = local_ordenado[n // 2][1]
        else:
            median = (local_ordenado[n // 2 - 1][1] + local_ordenado[n // 2][1]) / 2.0
        #Todas as imagens ordenadas pela entropia abaixo da medina são armezenadas nos indices_filtrados_da_Classe
        indices_filtrados_da_classe = [item[0] for item in local_ordenado if item[1] <= median]
        #as imagens abaixo da mediana são anexadas a cada nova interação em uma nova variavel.
        train_Xextend.extend([train_X[i] for i in indices_filtrados_da_classe])
        train_yextend.extend([train_y[i] for i in indices_filtrados_da_classe])
    test_Xextend = []
    test_yextend = []
    for label in range(totalClasses):
        indices_originais = np.where(test_y == label)[0] #pega os indices originais da classe em label range
        indicesDasImagensDaClasse = test_X[indices_originais] #Os indices das imagens da classe são armazenados
        #Calculo a entropia para cada imagem da classe e em seguida enumera os indices da classe.
        tuplas = [(indices_originais[index], entropia(img)) for index, img in enumerate(indicesDasImagensDaClasse)]
        #Ordena as tuplas com base no segundo elemento, ou seja com base na entropia
        local_ordenado = sorted(tuplas, key=lambda x: x[1])
        #pega o tamanho da classe ordenada e separa a mediana da classe.
        n = len(local_ordenado)
        if n % 2 == 1:
            median = local_ordenado[n // 2][1]
        else:
            median = (local_ordenado[n // 2 - 1][1] + local_ordenado[n // 2][1]) / 2.0
        #Todas as imagens ordenadas pela entropia abaixo da medina são armezenadas nos indices_filtrados_da_Classe
        indices_filtrados_da_classe = [item[0] for item in local_ordenado if item[1] <= median]
        #as imagens abaixo da mediana são anexadas a cada nova interação em uma nova variavel.
        test_Xextend.extend([test_X[i] for i in indices_filtrados_da_classe])
        test_yextend.extend([test_y[i] for i in indices_filtrados_da_classe])
    return np.array(train_Xextend), np.array(train_yextend), np.array(test_Xextend), np.array(test_yextend)

def carregarPeparar(train_X, train_y, Val_X, Val_y, test_X, test_y, d, c): # Carregamento e Tratamento
  train_X = train_X.reshape((train_X.shape[0], d, d, c))   # Ajuste da dimensão
  Val_X = Val_X.reshape((Val_X.shape[0], d, d, c))   # Ajuste da dimensão
  test_X = test_X.reshape((test_X.shape[0], d, d, c))    # Ajuste da dimensão
  train_X = train_X.astype("float")/255.0     # Normalização
  Val_X = Val_X.astype("float")/255.0     # Normalização
  test_X = test_X.astype("float")/255.0     # Normalização
  train_y = to_categorical(train_y, 10) #10 classes possiveis   -   # to Categorical para as classes
  Val_y = to_categorical(Val_y, 10) #10 classes possiveis   -   # to Categorical para as classes
  test_y = to_categorical(test_y, 10) #10 classes possiveis     -   # to Categorical para as classes
  return train_X, train_y, Val_X, Val_y, test_X, test_y

def carregarPepararValidacaoCruzada(train_X, train_y, test_X, test_y, d, c): # Carregamento e Tratamento
  train_X = train_X.reshape((train_X.shape[0], d, d, c))   # Ajuste da dimensão
  test_X = test_X.reshape((test_X.shape[0], d, d, c))    # Ajuste da dimensão
  train_X = train_X.astype("float")/255.0     # Normalização
  test_X = test_X.astype("float")/255.0     # Normalização
  train_y = to_categorical(train_y, 10) #10 classes possiveis   -   # to Categorical para as classes
  test_y = to_categorical(test_y, 10) #10 classes possiveis     -   # to Categorical para as classes
  return train_X, train_y, test_X, test_y

def avaliacao_statistica(train_X, train_y, Val_X, Val_y, test_X, test_y):
    # Para o conjunto de TREINAMENTO
    print("Dados de Treino")
    predictions_train = model.predict(train_X) # Previsões
    classePredita_train = np.argmax(predictions_train, axis=1)
    classeVerdadeira_train = np.argmax(train_y, axis=1)
    cm_train = confusion_matrix(classeVerdadeira_train, classePredita_train)

    disp = ConfusionMatrixDisplay(cm_train)
    disp.plot()
    plt.title('Confusion Matrix - Training')
    plt.show()

    acc_train = accuracy_score(classeVerdadeira_train, classePredita_train) * 100  # Acurácia
    print("Accuracy (Treinamento): ", '%.3f' % (acc_train))
    sens_train = recall_score(classeVerdadeira_train, classePredita_train, average='macro')  # Sensibilidade
    print("Recall (Treinamento): ", '%.3f' % (sens_train))
    loss_train = history.history['loss'][-1]
    print("Loss (Treinamento): ", '%.3f' % loss_train)

    print("--------------------------------------------------")
    print("Dados de Validação")
    # Para o conjunto de VALIDAÇÃO
    predictions_val = model.predict(Val_X) # Previsões
    classePredita_val = np.argmax(predictions_val, axis=1)
    classeVerdadeira_val = np.argmax(Val_y, axis=1)
    cm_val = confusion_matrix(classeVerdadeira_val, classePredita_val)

    disp = ConfusionMatrixDisplay(cm_val)
    disp.plot()
    plt.title('Confusion Matrix - Validation')
    plt.show()

    acc_val = accuracy_score(classeVerdadeira_val, classePredita_val) * 100  # Acurácia
    print("Accuracy (Validação): ", '%.3f' % (acc_val))
    sens_val = recall_score(classeVerdadeira_val, classePredita_val, average='macro')  # Sensibilidade
    print("Recall (Validação): ", '%.3f' % (sens_val))
    loss_val = history.history['val_loss'][-1]
    print("Loss (Validação): ", '%.3f' % loss_val)

    print("--------------------------------------------------")
    print("Dados de Teste")
    # Para o conjunto de TESTE
    predictions_test = model.predict(test_X) # Previsões
    classePredita_test = np.argmax(predictions_test, axis=1)
    classeVerdadeira_test = np.argmax(test_y, axis=1)
    cm_test = confusion_matrix(classeVerdadeira_test, classePredita_test)

    disp = ConfusionMatrixDisplay(cm_test)
    disp.plot()
    plt.title('Confusion Matrix - Test')
    plt.show()

    acc_test = accuracy_score(classeVerdadeira_test, classePredita_test) * 100  # Acurácia
    print("Accuracy (Teste): ", '%.3f' % (acc_test))
    sens_test = recall_score(classeVerdadeira_test, classePredita_test, average='macro')  # Sensibilidade
    print("Recall (Teste): ", '%.3f' % (sens_test))
    loss_test = history.history['val_loss'][-1]
    print("Loss (Teste): ", '%.3f' % loss_test)

def plotAcuraciaLoss(history): #plots
  plt.figure(figsize=(12, 5))
  plt.subplot(1, 2, 1)
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('Model Accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  plt.subplot(1, 2, 2)
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model Loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  plt.tight_layout()

def plotarDadosTrain(index_inicio, qtd, train_X): # Exemplo: apartir do indice N, exibe K imagens do conjunto de dados, D
  base = 330
  import matplotlib.pyplot as plt
  for i in range(index_inicio, index_inicio + qtd):
      plt.subplot(base + 1 + (i - index_inicio))
      plt.imshow(train_X[i], cmap=plt.get_cmap('gray'))
  plt.show()
  print(train_y[index_inicio:index_inicio + qtd])

def entHistogram(testar, num):
  import matplotlib.pyplot as plt
  if num == 1:
    title = 'Class histogram train_y'
  elif num == 2:
    title = 'Class histogram Val_y'
  elif num == 3:
    title = 'Class histogram test_y'
  plt.hist(testar, bins=10, edgecolor='black', alpha=0.7)
  plt.title(title)
  plt.xlabel('Value')
  plt.ylabel('Frequency')
  plt.show()

def tempDecorrido(inicio, importacaoF, padrozinacaoF, criacaoModeloF, treinamentoF, entropiaF = None): # x = time.time() || inicio, importacaoF, padrozinacaoF, criacaoModeloF, treinamentoF, entropiaF
  elapsed_time = [["Carregar DataSet", 0], ["Filtro Entropia", 0], ["Pre-processamento", 0], ["Criação do Modelo", 0], ["Treinamento", 0], ["Inicio à Fim Execução" , 0]]
  elapsed_time[0][1] = importacaoF - inicio  # tempo de inicio de execução até o final da importação do dataset
  if (entropiaF != None): elapsed_time[1][1] = entropiaF - importacaoF  # Tempo gasto Inicio é importaçãoF - Execução da Entropia
  elapsed_time[2][1] = padrozinacaoF - importacaoF  # pega o tempo da entropia e subtrai do tempo apos a padronização para verificar quanto tempo padronizacao demorou
  elapsed_time[3][1] = criacaoModeloF - padrozinacaoF  # tempo da padronizacao - tempo apos a criacao do modelo para verificar tempo decorido
  elapsed_time[4][1] = treinamentoF - criacaoModeloF  # tempo inicial é marcado pela criacaoModeloF - tempoTreinamento que marca o momento que treinamento terminou
  elapsed_time[5][1] = treinamentoF - inicio #tempo total de execução inicio| importações até final da Execução | treinamento
  for index, tempo in enumerate(elapsed_time):
    if entropiaF is None and index == 1:
      continue
    print("{}: {:.4f}".format(tempo[0], round(tempo[1], 2)) + str(" em milissegundos"))

In [ ]:
def criacaoModeloF1(): # Criação do Modelo
  model = Sequential()
  model.add(Conv2D(32, (3,3), activation = 'relu', kernel_initializer = 'he_uniform', input_shape = (28, 28, 1)))
  model.add(MaxPooling2D((2, 2)))

  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(Dropout(0.4))
  model.add(BatchNormalization())
  model.add(MaxPooling2D((2, 2)))

  model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D((2, 2)))

  model.add(Flatten())
  model.add(Dense(128, activation = 'relu', kernel_initializer = 'he_uniform'))
  model.add(Dropout(0.5))
  model.add(Dense(10, activation = 'softmax'))  # classificação 10 categorias

  opt = SGD(learning_rate=0.01, momentum =0.9) #copilaçãoModelo
  model.compile(optimizer=opt, loss = 'categorical_crossentropy', metrics= ['accuracy'])
  return model

## [MNIST] Full Dataset Training:
  * Without entropy selection
  * Full Dataset


In [ ]:
inicio = time.time() #tempo de inicio
(train_X, train_y), (test_X, test_y) = mnist.load_data() # Carregar o conjunto de dados # Importar e Treinar
train_X, Val_X, train_y, Val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y) # divisao do conjunto de dados
entHistogram(train_y, 1)
entHistogram(Val_y, 2)
entHistogram(test_y, 3)
print(len(train_X))
print(len(train_y))
print(len(test_X))
print(len(test_y))

In [ ]:
from sklearn.model_selection import train_test_split

inicio = time.time() #tempo de inicio
(train_X, train_y), (test_X, test_y) = mnist.load_data() # Carregar o conjunto de dados # Importar e Treinar
importacaoF = time.time() #tempo de importacao
train_X, Val_X, train_y, Val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y) # divisao do conjunto de dados
train_X, train_y, Val_X, Val_y, test_X, test_y = carregarPeparar(train_X, train_y, Val_X, Val_y, test_X, test_y, 28, 1) #padronização e categorização
padrozinacaoF = time.time() #tempo de padronizacao
model = criacaoModeloF1() # instanciando o modelo
criacaoModeloF = time.time() #tempo de criacao do modelo
history = model.fit(train_X, train_y, epochs = 10, batch_size = 32, validation_data=(Val_X, Val_y), verbose = 1) #treinamento
treinamentoF = time.time() #tempo final
tempDecorrido(inicio, importacaoF, padrozinacaoF, criacaoModeloF, treinamentoF) #inicio, importacaoF, padrozinacaoF, criacaoModeloF, treinamentoF, entropiaF
print("train_X: ",len(train_X))
print("train_y: ",len(train_y))
print("Val_X: ",len(Val_X))
print("Val_y: ",len(Val_y))
print("test_X: ",len(test_X))
print("test_y: ",len(test_y))
avaliacao_statistica(train_X, train_y, Val_X, Val_y, test_X, test_y)
plotAcuraciaLoss(history)

## [MNIST] Random and Stratified Training:
  * Without entropy selection
  * Stratified dataset divided proportionally in half


In [ ]:
from sklearn.model_selection import train_test_split
(train_X, train_y), (test_X, test_y) = mnist.load_data() # Carregar o conjunto de dados # Importar e Treinar
partial_train_X, _, partial_train_y, _ = train_test_split(train_X, train_y, test_size=0.5, random_state=42, stratify=train_y)
train_X, Val_X, train_y, Val_y = train_test_split(partial_train_X, partial_train_y, test_size=(0.2), random_state=42, stratify=partial_train_y)
entHistogram(train_y, 1)
entHistogram(Val_y, 2)
entHistogram(test_y, 3)
print(len(train_X))
print(len(train_y))
print(len(test_X))
print(len(test_y))

In [ ]:
from sklearn.model_selection import train_test_split

inicio = time.time() #tempo de inicio
(train_X, train_y), (test_X, test_y) = mnist.load_data() # Carregar o conjunto de dados # Importar e Treinar
importacaoF = time.time() #tempo de importacao

partial_train_X, _, partial_train_y, _ = train_test_split(train_X, train_y, test_size=0.5, random_state=42, stratify=train_y)
train_X, Val_X, train_y, Val_y = train_test_split(partial_train_X, partial_train_y, test_size=(0.2), random_state=42, stratify=partial_train_y)

train_X, train_y, Val_X, Val_y, test_X, test_y = carregarPeparar(train_X, train_y, Val_X, Val_y, test_X, test_y, 28, 1) #padronização e categorização
padrozinacaoF = time.time() #tempo de padronizacao
model = criacaoModeloF1() # instanciando o modelo
criacaoModeloF = time.time() #tempo de criacao do modelo
history = model.fit(train_X, train_y, epochs = 10, batch_size = 32, validation_data=(Val_X, Val_y), verbose = 1) #treinamento
treinamentoF = time.time() #tempo final

tempDecorrido(inicio, importacaoF, padrozinacaoF, criacaoModeloF, treinamentoF) #inicio, importacaoF, padrozinacaoF, criacaoModeloF, treinamentoF, entropiaF
print("train_X: ",len(train_X))
print("train_y: ",len(train_y))
print("Val_X: ",len(Val_X))
print("Val_y: ",len(Val_y))
print("test_X: ",len(test_X))
print("test_y: ",len(test_y))
avaliacao_statistica(train_X, train_y, Val_X, Val_y, test_X, test_y)
plotAcuraciaLoss(history)

## [MNIST] Entropy Filter:
  * Filtered Dataset
  * Low Entropy in training data causing "noise" in validation.


In [ ]:
(train_X, train_y), (test_X, test_y) = mnist.load_data() # Carregar o conjunto de dados # Importar e Treinar
train_X, Val_X, train_y, Val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y) # divisao do conjunto de dados
train_X, train_y = filtrar_entropia_classe(train_X, train_y,10)
entHistogram(train_y, 1)
entHistogram(Val_y, 2)
entHistogram(test_y, 3)
print(len(train_X))
print(len(train_y))
print(len(test_X))
print(len(test_y))

In [ ]:
inicio = time.time() #tempo de inicio
(train_X, train_y), (test_X, test_y) = mnist.load_data() # Carregar o conjunto de dados # Importar e Treinar
importacaoF = time.time() #tempo de importacao

train_X, Val_X, train_y, Val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y) # divisao do conjunto de dados
train_X, train_y = filtrar_entropia_classe(train_X, train_y,10)

entropiaF = time.time() #tempo de filtragem entropia
train_X, train_y, Val_X, Val_y, test_X, test_y = carregarPeparar(train_X, train_y, Val_X, Val_y, test_X, test_y, 28, 1) #padronização e categorização
padrozinacaoF = time.time() #tempo de padronizacao
model = criacaoModeloF1() # instanciando o modelo
criacaoModeloF = time.time() #tempo de criacao do modelo
history = model.fit(train_X, train_y, epochs = 10, batch_size = 32, validation_data=(Val_X, Val_y), verbose = 1) #treinamento
treinamentoF = time.time() #tempo final

tempDecorrido(inicio, importacaoF, padrozinacaoF, criacaoModeloF, treinamentoF, entropiaF) #inicio, importacaoF, padrozinacaoF, criacaoModeloF, treinamentoF, entropiaF
print("train_X: ",len(train_X))
print("train_y: ",len(train_y))
print("Val_X: ",len(Val_X))
print("Val_y: ",len(Val_y))
print("test_X: ",len(test_X))
print("test_y: ",len(test_y))
avaliacao_statistica(train_X, train_y, Val_X, Val_y, test_X, test_y)
plotAcuraciaLoss(history)

## [MNIST] Entropy Filter:
  * Filtered Dataset
  * Low Entropy in both training and validation data.


In [ ]:
(train_X, train_y), (test_X, test_y) = mnist.load_data() # Carregar o conjunto de dados # Importar e Treinar
train_X, train_y = filtrar_entropia_classe(train_X, train_y,10)
train_X, Val_X, train_y, Val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y) # divisao do conjunto de dados
entHistogram(train_y, 1)
entHistogram(Val_y, 2)
entHistogram(test_y, 3)
print(len(train_X))
print(len(train_y))
print(len(test_X))
print(len(test_y))

In [ ]:
inicio = time.time() #tempo de inicio
(train_X, train_y), (test_X, test_y) = mnist.load_data() # Carregar o conjunto de dados # Importar e Treinar
importacaoF = time.time() #tempo de importacao

train_X, train_y = filtrar_entropia_classe(train_X, train_y,10)
train_X, Val_X, train_y, Val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y) # divisao do conjunto de dados

entropiaF = time.time() #tempo de filtragem entropia
train_X, train_y, Val_X, Val_y, test_X, test_y = carregarPeparar(train_X, train_y, Val_X, Val_y, test_X, test_y, 28, 1) #padronização e categorização
padrozinacaoF = time.time() #tempo de padronizacao
model = criacaoModeloF1() # instanciando o modelo
criacaoModeloF = time.time() #tempo de criacao do modelo
history = model.fit(train_X, train_y, epochs = 10, batch_size = 32, validation_split=0.2, verbose = 1) #treinamento
treinamentoF = time.time() #tempo final

tempDecorrido(inicio, importacaoF, padrozinacaoF, criacaoModeloF, treinamentoF, entropiaF) #inicio, importacaoF, padrozinacaoF, criacaoModeloF, treinamentoF, entropiaF
print("train_X: ",len(train_X))
print("train_y: ",len(train_y))
print("Val_X: ",len(Val_X))
print("Val_y: ",len(Val_y))
print("test_X: ",len(test_X))
print("test_y: ",len(test_y))
avaliacao_statistica(train_X, train_y, Val_X, Val_y, test_X, test_y)
plotAcuraciaLoss(history)

## [MNIST] Entropy Filter:
  * Filtered Dataset
  * Low Entropy in both training and validation data.
  * Cross-validation


In [ ]:
from sklearn.model_selection import KFold

def crossValidation(train_X,train_y,n_folds = 5):
    #divide seus dados em k folds, e treina no k−1 e valida em 1 fold, então a proporção de dados usados para validação é 1/k.
    #2 folds (50% de validação): Treina em 50% dos dados e valida nos outros 50%.
    #5 folds (20% de validação): Treina em 80% dos dados e valida em 20%.
    #10 folds (10% de validação): Treina em 90% dos dados e valida em 10%.
    n_fold = 5 if n_folds == 0 else n_folds

    kf = KFold(n_splits = n_folds, shuffle = True, random_state = 42)  #divisão do conjunto de dados
    fold_no = 1
    for train_index, val_index in kf.split(train_X):
        train_data, val_data = train_X[train_index], train_X[val_index]
        train_label, val_label = train_y[train_index], train_y[val_index]

        print(f"Treinando a rede no folder numero: {fold_no}")
        model = criacaoModeloF1() # instanciando o modelo
        inicio = time.time() #tempo de inicio
        history = model.fit(train_data, train_label, epochs = 10, batch_size = 32, verbose = 1, validation_data = (val_data, val_label))
        treinamentoF = time.time() #tempo final
        tempoTreino = round(treinamentoF - inicio,2)
        scores = model.evaluate(val_data, val_label, verbose = 1)
        print(f"Num do Folder: {fold_no}: Taxa de Loss: {scores[0]}, Taxa de Acuracia: {scores[1]*100:.2f}%, Tamanho do Conjunto de Dados: {len(train_data)}, Tempo de Treino: {tempoTreino}")
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title(f'Model Accuracy: {fold_no}')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title(f'Model Loss: {fold_no}')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.tight_layout()

        # Para o conjunto de TREINAMENTO
        print("Dados de Treino")
        predictions_train = model.predict(train_data) # Previsões
        classePredita_train = np.argmax(predictions_train, axis=1)
        classeVerdadeira_train = np.argmax(train_label, axis=1)
        cm_train = confusion_matrix(classeVerdadeira_train, classePredita_train)
        disp = ConfusionMatrixDisplay(cm_train)
        disp.plot()
        plt.title('Confusion Matrix - Training')
        plt.show()

        acc_train = accuracy_score(classeVerdadeira_train, classePredita_train) * 100  # Acurácia
        print("Accuracy (Treinamento): ", '%.3f' % (acc_train))
        sens_train = recall_score(classeVerdadeira_train, classePredita_train, average='macro')  # Sensibilidade
        print("Recall (Treinamento): ", '%.3f' % (sens_train))
        loss_train = history.history['loss'][-1]
        print("Loss (Treinamento): ", '%.3f' % loss_train)

        print("--------------------------------------------------")
        print("Dados de Validação")
        # Para o conjunto de VALIDAÇÃO
        predictions_val = model.predict(val_data) # Previsões
        classePredita_val = np.argmax(predictions_val, axis=1)
        classeVerdadeira_val = np.argmax(val_label, axis=1)
        cm_val = confusion_matrix(classeVerdadeira_val, classePredita_val)
        disp = ConfusionMatrixDisplay(cm_val)
        disp.plot()
        plt.title('Confusion Matrix - Validation')
        plt.show()

        acc_val = accuracy_score(classeVerdadeira_val, classePredita_val) * 100  # Acurácia
        print("Accuracy (Validação): ", '%.3f' % (acc_val))
        sens_val = recall_score(classeVerdadeira_val, classePredita_val, average='macro')  # Sensibilidade
        print("Recall (Validação): ", '%.3f' % (sens_val))
        loss_val = history.history['val_loss'][-1]
        print("Loss (Validação): ", '%.3f' % loss_val)

        fold_no += 1
        print("\n")


#carregar os dados
(train_X, train_y), (test_X, test_y) = mnist.load_data() # Carregar o conjunto de dados # Importar e Treinar
train_X, train_y, test_X, test_y = filtrar_entropia_median_train_teste(train_X, train_y, test_X, test_y, 10) #novo filtro da reunião 03-10-2023
train_X, train_y, test_X, test_y = carregarPepararValidacaoCruzada(train_X, train_y, test_X, test_y, 28, 1) #padronização e categorização
train_X = np.concatenate([train_X, test_X], axis=0)
train_y = np.concatenate([train_y, test_y], axis=0)

#criar/instanciar o modelo para validação cruzada
crossValidation(train_X,train_y, 5)


# FASHION MNIST
* An alternative to MNIST, containing 70,000 grayscale images of 28x28 pixels of 10 categories of clothing items, such as t-shirts, pants, and shoes.


## Creation and definition of all functions

1. Entropy Filter
2. Loading, Preprocessing, and Handling
3. Model Creation
4. Statistical Predictions
5. Accuracy and Loss Plot


In [ ]:
from keras.datasets import mnist
import numpy as np
from scipy.stats import entropy
import matplotlib.pyplot as plt
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Sequential
from keras.optimizers import SGD
from sklearn import metrics
from sklearn.metrics import accuracy_score, recall_score
from keras.utils import to_categorical
from keras.layers import Dropout
from keras.layers import BatchNormalization
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import time
from sklearn.model_selection import train_test_split

def entropia(pk, base=10, max=1):
    pk = pk / np.sum(pk)
    pk = pk[pk > 0]
    result = -np.sum(pk * np.log(pk) / np.log(base))
    return result

def filtrar_entropia_classe(train_X, train_y, totalClasses):  # filtro de entropia
    train_Xextend = []
    train_yextend = []
    for label in range(totalClasses):
        indices_originais = np.where(train_y == label)[0] #pega os indices originais da classe em label range
        indicesDasImagensDaClasse = train_X[indices_originais] #Os indices das imagens da classe são armazenados
        #Calculo a entropia para cada imagem da classe e em seguida enumera os indices da classe.
        tuplas = [(indices_originais[index], entropia(img)) for index, img in enumerate(indicesDasImagensDaClasse)]
        #Ordena as tuplas com base no segundo elemento, ou seja com base na entropia
        local_ordenado = sorted(tuplas, key=lambda x: x[1])
        #pega o tamanho da classe ordenada e separa a mediana da classe.
        n = len(local_ordenado)
        if n % 2 == 1:
            median = local_ordenado[n // 2][1]
        else:
            median = (local_ordenado[n // 2 - 1][1] + local_ordenado[n // 2][1]) / 2.0
        #Todas as imagens ordenadas pela entropia abaixo da medina são armezenadas nos indices_filtrados_da_Classe
        indices_filtrados_da_classe = [item[0] for item in local_ordenado if item[1] <= median]
        #as imagens abaixo da mediana são anexadas a cada nova interação em uma nova variavel.
        train_Xextend.extend([train_X[i] for i in indices_filtrados_da_classe])
        train_yextend.extend([train_y[i] for i in indices_filtrados_da_classe])
    return np.array(train_Xextend), np.array(train_yextend)

def filtrar_entropia_median_train_teste(train_X, train_y, test_X, test_y, totalClasses):  # filtro de entropia
    train_Xextend = []
    train_yextend = []
    for label in range(totalClasses):
        indices_originais = np.where(train_y == label)[0] #pega os indices originais da classe em label range
        indicesDasImagensDaClasse = train_X[indices_originais] #Os indices das imagens da classe são armazenados
        #Calculo a entropia para cada imagem da classe e em seguida enumera os indices da classe.
        tuplas = [(indices_originais[index], entropia(img)) for index, img in enumerate(indicesDasImagensDaClasse)]
        #Ordena as tuplas com base no segundo elemento, ou seja com base na entropia
        local_ordenado = sorted(tuplas, key=lambda x: x[1])
        #pega o tamanho da classe ordenada e separa a mediana da classe.
        n = len(local_ordenado)
        if n % 2 == 1:
            median = local_ordenado[n // 2][1]
        else:
            median = (local_ordenado[n // 2 - 1][1] + local_ordenado[n // 2][1]) / 2.0
        #Todas as imagens ordenadas pela entropia abaixo da medina são armezenadas nos indices_filtrados_da_Classe
        indices_filtrados_da_classe = [item[0] for item in local_ordenado if item[1] <= median]
        #as imagens abaixo da mediana são anexadas a cada nova interação em uma nova variavel.
        train_Xextend.extend([train_X[i] for i in indices_filtrados_da_classe])
        train_yextend.extend([train_y[i] for i in indices_filtrados_da_classe])
    test_Xextend = []
    test_yextend = []
    for label in range(totalClasses):
        indices_originais = np.where(test_y == label)[0] #pega os indices originais da classe em label range
        indicesDasImagensDaClasse = test_X[indices_originais] #Os indices das imagens da classe são armazenados
        #Calculo a entropia para cada imagem da classe e em seguida enumera os indices da classe.
        tuplas = [(indices_originais[index], entropia(img)) for index, img in enumerate(indicesDasImagensDaClasse)]
        #Ordena as tuplas com base no segundo elemento, ou seja com base na entropia
        local_ordenado = sorted(tuplas, key=lambda x: x[1])
        #pega o tamanho da classe ordenada e separa a mediana da classe.
        n = len(local_ordenado)
        if n % 2 == 1:
            median = local_ordenado[n // 2][1]
        else:
            median = (local_ordenado[n // 2 - 1][1] + local_ordenado[n // 2][1]) / 2.0
        #Todas as imagens ordenadas pela entropia abaixo da medina são armezenadas nos indices_filtrados_da_Classe
        indices_filtrados_da_classe = [item[0] for item in local_ordenado if item[1] <= median]
        #as imagens abaixo da mediana são anexadas a cada nova interação em uma nova variavel.
        test_Xextend.extend([test_X[i] for i in indices_filtrados_da_classe])
        test_yextend.extend([test_y[i] for i in indices_filtrados_da_classe])
    return np.array(train_Xextend), np.array(train_yextend), np.array(test_Xextend), np.array(test_yextend)

def carregarPeparar(train_X, train_y, Val_X, Val_y, test_X, test_y, d, c): # Carregamento e Tratamento
  train_X = train_X.reshape((train_X.shape[0], d, d, c))   # Ajuste da dimensão
  Val_X = Val_X.reshape((Val_X.shape[0], d, d, c))   # Ajuste da dimensão
  test_X = test_X.reshape((test_X.shape[0], d, d, c))    # Ajuste da dimensão
  train_X = train_X.astype("float")/255.0     # Normalização
  Val_X = Val_X.astype("float")/255.0     # Normalização
  test_X = test_X.astype("float")/255.0     # Normalização
  train_y = to_categorical(train_y, 10) #10 classes possiveis   -   # to Categorical para as classes
  Val_y = to_categorical(Val_y, 10) #10 classes possiveis   -   # to Categorical para as classes
  test_y = to_categorical(test_y, 10) #10 classes possiveis     -   # to Categorical para as classes
  return train_X, train_y, Val_X, Val_y, test_X, test_y

def carregarPepararValidacaoCruzada(train_X, train_y, test_X, test_y, d, c): # Carregamento e Tratamento
  train_X = train_X.reshape((train_X.shape[0], d, d, c))   # Ajuste da dimensão
  test_X = test_X.reshape((test_X.shape[0], d, d, c))    # Ajuste da dimensão
  train_X = train_X.astype("float")/255.0     # Normalização
  test_X = test_X.astype("float")/255.0     # Normalização
  train_y = to_categorical(train_y, 10) #10 classes possiveis   -   # to Categorical para as classes
  test_y = to_categorical(test_y, 10) #10 classes possiveis     -   # to Categorical para as classes
  return train_X, train_y, test_X, test_y

def avaliacao_statistica(train_X, train_y, Val_X, Val_y, test_X, test_y):
    # Para o conjunto de TREINAMENTO
    print("Dados de Treino")
    predictions_train = model.predict(train_X) # Previsões
    classePredita_train = np.argmax(predictions_train, axis=1)
    classeVerdadeira_train = np.argmax(train_y, axis=1)
    cm_train = confusion_matrix(classeVerdadeira_train, classePredita_train)

    disp = ConfusionMatrixDisplay(cm_train)
    disp.plot()
    plt.title('Confusion Matrix - Training')
    plt.show()

    acc_train = accuracy_score(classeVerdadeira_train, classePredita_train) * 100  # Acurácia
    print("Accuracy (Treinamento): ", '%.3f' % (acc_train))
    sens_train = recall_score(classeVerdadeira_train, classePredita_train, average='macro')  # Sensibilidade
    print("Recall (Treinamento): ", '%.3f' % (sens_train))
    loss_train = history.history['loss'][-1]
    print("Loss (Treinamento): ", '%.3f' % loss_train)

    print("--------------------------------------------------")
    print("Dados de Validação")
    # Para o conjunto de VALIDAÇÃO
    predictions_val = model.predict(Val_X) # Previsões
    classePredita_val = np.argmax(predictions_val, axis=1)
    classeVerdadeira_val = np.argmax(Val_y, axis=1)
    cm_val = confusion_matrix(classeVerdadeira_val, classePredita_val)

    disp = ConfusionMatrixDisplay(cm_val)
    disp.plot()
    plt.title('Confusion Matrix - Validation')
    plt.show()

    acc_val = accuracy_score(classeVerdadeira_val, classePredita_val) * 100  # Acurácia
    print("Accuracy (Validação): ", '%.3f' % (acc_val))
    sens_val = recall_score(classeVerdadeira_val, classePredita_val, average='macro')  # Sensibilidade
    print("Recall (Validação): ", '%.3f' % (sens_val))
    loss_val = history.history['val_loss'][-1]
    print("Loss (Validação): ", '%.3f' % loss_val)

    print("--------------------------------------------------")
    print("Dados de Teste")
    # Para o conjunto de TESTE
    predictions_test = model.predict(test_X) # Previsões
    classePredita_test = np.argmax(predictions_test, axis=1)
    classeVerdadeira_test = np.argmax(test_y, axis=1)
    cm_test = confusion_matrix(classeVerdadeira_test, classePredita_test)

    disp = ConfusionMatrixDisplay(cm_test)
    disp.plot()
    plt.title('Confusion Matrix - Test')
    plt.show()

    acc_test = accuracy_score(classeVerdadeira_test, classePredita_test) * 100  # Acurácia
    print("Accuracy (Teste): ", '%.3f' % (acc_test))
    sens_test = recall_score(classeVerdadeira_test, classePredita_test, average='macro')  # Sensibilidade
    print("Recall (Teste): ", '%.3f' % (sens_test))
    loss_test = history.history['val_loss'][-1]
    print("Loss (Teste): ", '%.3f' % loss_test)

def plotAcuraciaLoss(history): #plots
  plt.figure(figsize=(12, 5))
  plt.subplot(1, 2, 1)
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('Model Accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  plt.subplot(1, 2, 2)
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model Loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  plt.tight_layout()

def plotarDadosTrain(index_inicio, qtd, train_X): # Exemplo: apartir do indice N, exibe K imagens do conjunto de dados, D
  base = 330
  import matplotlib.pyplot as plt
  for i in range(index_inicio, index_inicio + qtd):
      plt.subplot(base + 1 + (i - index_inicio))
      plt.imshow(train_X[i], cmap=plt.get_cmap('gray'))
  plt.show()
  print(train_y[index_inicio:index_inicio + qtd])

def entHistogram(testar, num):
  import matplotlib.pyplot as plt
  if num == 1:
    title = 'Class histogram train_y'
  elif num == 2:
    title = 'Class histogram Val_y'
  elif num == 3:
    title = 'Class histogram test_y'
  plt.hist(testar, bins=10, edgecolor='black', alpha=0.7)
  plt.title(title)
  plt.xlabel('Value')
  plt.ylabel('Frequency')
  plt.show()

def tempDecorrido(inicio, importacaoF, padrozinacaoF, criacaoModeloF, treinamentoF, entropiaF = None): # x = time.time() || inicio, importacaoF, padrozinacaoF, criacaoModeloF, treinamentoF, entropiaF
  elapsed_time = [["Carregar DataSet", 0], ["Filtro Entropia", 0], ["Pre-processamento", 0], ["Criação do Modelo", 0], ["Treinamento", 0], ["Inicio à Fim Execução" , 0]]
  elapsed_time[0][1] = importacaoF - inicio  # tempo de inicio de execução até o final da importação do dataset
  if (entropiaF != None): elapsed_time[1][1] = entropiaF - importacaoF  # Tempo gasto Inicio é importaçãoF - Execução da Entropia
  elapsed_time[2][1] = padrozinacaoF - importacaoF  # pega o tempo da entropia e subtrai do tempo apos a padronização para verificar quanto tempo padronizacao demorou
  elapsed_time[3][1] = criacaoModeloF - padrozinacaoF  # tempo da padronizacao - tempo apos a criacao do modelo para verificar tempo decorido
  elapsed_time[4][1] = treinamentoF - criacaoModeloF  # tempo inicial é marcado pela criacaoModeloF - tempoTreinamento que marca o momento que treinamento terminou
  elapsed_time[5][1] = treinamentoF - inicio #tempo total de execução inicio| importações até final da Execução | treinamento
  for index, tempo in enumerate(elapsed_time):
    if entropiaF is None and index == 1:
      continue
    print("{}: {:.4f}".format(tempo[0], round(tempo[1], 2)) + str(" em milissegundos"))

In [ ]:
from numpy import mean, std
from matplotlib import pyplot
from sklearn.model_selection import KFold
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.optimizers import SGD
import numpy as np
from scipy.stats import entropy
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split

def criacaoModeloFashion(): # Criação do Modelo
  model = Sequential()
  model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(28, 28, 1)))
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
  model.add(Dropout(rate=0.45))
  model.add(Flatten())
  model.add(Dense(units=128, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dropout(rate=0.25))
  model.add(Dense(units=10, activation='softmax'))
  opt = SGD(learning_rate=0.01, momentum =0.9)
  model.compile(optimizer=opt, loss = 'categorical_crossentropy', metrics= ['accuracy'])
  return model

## [FASHION MNIST] Full Dataset Training:
  * Without entropy selection
  * Full Dataset


In [ ]:
import tensorflow as tf
(train_X, train_y), (test_X, test_y) = tf.keras.datasets.fashion_mnist.load_data() # Carregar o conjunto de dados # Importar e Treinar
train_X, Val_X, train_y, Val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y) # divisao do conjunto de dados
entHistogram(train_y, 1)
entHistogram(Val_y, 2)
entHistogram(test_y, 3)
print(len(train_X))
print(len(train_y))
print(len(test_X))
print(len(test_y))

In [ ]:
(train_X, train_y), (test_X, test_y) = tf.keras.datasets.fashion_mnist.load_data()
train_X, Val_X, train_y, Val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y) # divisao do conjunto de dados
train_X, train_y, Val_X, Val_y, test_X, test_y = carregarPeparar(train_X, train_y, Val_X, Val_y, test_X, test_y, 28, 1) #padronização e categorização

# Criaçaõ do Modelo
model = criacaoModeloFashion() # instanciando o modelo

# Treinamento
tempoIni = time.time() #tempo de filtragem entropia
history = model.fit(train_X, train_y, epochs = 10, batch_size = 128, validation_data=(Val_X, Val_y), verbose = 1)
tempoFim = time.time() #tempo de filtragem entropia
tempoTotal = round(tempoFim - tempoIni,2)

print("train_X: ",len(train_X))
print("train_y: ",len(train_y))
print("Val_X: ",len(Val_X))
print("Val_y: ",len(Val_y))
print("test_X: ",len(test_X))
print("test_y: ",len(test_y))
print("tempoTotal de treinamento: ", tempoTotal)
avaliacao_statistica(train_X, train_y, Val_X, Val_y, test_X, test_y)
plotAcuraciaLoss(history)

## [Fashion MNIST] Random and Stratified Training:
  * Without entropy selection
  * Stratified dataset divided proportionally in half


In [ ]:
from sklearn.model_selection import train_test_split
(train_X, train_y), (test_X, test_y) = tf.keras.datasets.fashion_mnist.load_data()
partial_train_X, _, partial_train_y, _ = train_test_split(train_X, train_y, test_size=0.5, random_state=42, stratify=train_y)
train_X, Val_X, train_y, Val_y = train_test_split(partial_train_X, partial_train_y, test_size=(0.2), random_state=42, stratify=partial_train_y)
entHistogram(train_y, 1)
entHistogram(Val_y, 2)
entHistogram(test_y, 3)
print(len(train_X))
print(len(train_y))
print(len(test_X))
print(len(test_y))

In [ ]:
(train_X, train_y), (test_X, test_y) = tf.keras.datasets.fashion_mnist.load_data()
partial_train_X, _, partial_train_y, _ = train_test_split(train_X, train_y, test_size=0.5, random_state=42, stratify=train_y)
train_X, Val_X, train_y, Val_y = train_test_split(partial_train_X, partial_train_y, test_size=(0.2), random_state=42, stratify=partial_train_y)
train_X, train_y, Val_X, Val_y, test_X, test_y = carregarPeparar(train_X, train_y, Val_X, Val_y, test_X, test_y, 28, 1) #padronização e categorização

# Criaçaõ do Modelo
model = criacaoModeloFashion() # instanciando o modelo

# Treinamento
tempoIni = time.time() #tempo de filtragem entropia
history = model.fit(train_X, train_y, epochs = 10, batch_size = 128, validation_data=(Val_X, Val_y), verbose = 1)
tempoFim = time.time() #tempo de filtragem entropia
tempoTotal = round(tempoFim - tempoIni,2)

print("train_X: ",len(train_X))
print("train_y: ",len(train_y))
print("Val_X: ",len(Val_X))
print("Val_y: ",len(Val_y))
print("test_X: ",len(test_X))
print("test_y: ",len(test_y))
print("tempoTotal de treinamento: ", tempoTotal)
avaliacao_statistica(train_X, train_y, Val_X, Val_y, test_X, test_y)
plotAcuraciaLoss(history)

## [Fashion MNIST] Entropy Filter:
  * Filtered Dataset
  * Low Entropy in training data causing "noise" in validation.


In [ ]:
(train_X, train_y), (test_X, test_y) =  tf.keras.datasets.fashion_mnist.load_data() # Carregar o conjunto de dados # Importar e Treinar
train_X, Val_X, train_y, Val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y) # divisao do conjunto de dados
train_X, train_y = filtrar_entropia_classe(train_X, train_y,10)
entHistogram(train_y, 1)
entHistogram(Val_y, 2)
entHistogram(test_y, 3)
print(len(train_X))
print(len(train_y))
print(len(test_X))
print(len(test_y))

In [ ]:
(train_X, train_y), (test_X, test_y) = tf.keras.datasets.fashion_mnist.load_data()


train_X, Val_X, train_y, Val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y) # divisao do conjunto de dados
tempoIniCalEntropia = time.time() #tempo de filtragem entropia
train_X, train_y = filtrar_entropia_classe(train_X, train_y,10)
tempoFimCalEntropia = time.time() #tempo de filtragem entropia
train_X, train_y, Val_X, Val_y, test_X, test_y = carregarPeparar(train_X, train_y, Val_X, Val_y, test_X, test_y, 28, 1) #padronização e categorização

# Criaçaõ do Modelo
model = criacaoModeloFashion() # instanciando o modelo

# Treinamento
tempoIni = time.time() #tempo de filtragem entropia
history = model.fit(train_X, train_y, epochs = 10, batch_size = 128, validation_data=(Val_X, Val_y), verbose = 1)
tempoFim = time.time() #tempo de filtragem entropia
tempoTotal = round(tempoFim - tempoIni,2)
tempoTotalCalEntropia = round(tempoFimCalEntropia - tempoIniCalEntropia,2)

print("train_X: ",len(train_X))
print("train_y: ",len(train_y))
print("Val_X: ",len(Val_X))
print("Val_y: ",len(Val_y))
print("test_X: ",len(test_X))
print("test_y: ",len(test_y))
print("tempoTotal de treinamento: ", tempoTotal)
print("tempo Total CalEntropia: ", tempoTotalCalEntropia)

avaliacao_statistica(train_X, train_y, Val_X, Val_y, test_X, test_y)
plotAcuraciaLoss(history)

## [FASHION MNIST] Entropy Filter:
  * Filtered Dataset
  * Low Entropy in both training and validation data.


In [ ]:
(train_X, train_y), (test_X, test_y) =  tf.keras.datasets.fashion_mnist.load_data() # Carregar o conjunto de dados # Importar e Treinar
train_X, train_y = filtrar_entropia_classe(train_X, train_y,10)
train_X, Val_X, train_y, Val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y) # divisao do conjunto de dados
entHistogram(train_y, 1)
entHistogram(Val_y, 2)
entHistogram(test_y, 3)
print(len(train_X))
print(len(train_y))
print(len(test_X))
print(len(test_y))

In [ ]:
(train_X, train_y), (test_X, test_y) = tf.keras.datasets.fashion_mnist.load_data()

tempoIniCalEntropia = time.time() #tempo de filtragem entropia
train_X, train_y = filtrar_entropia_classe(train_X, train_y,10)
tempoFimCalEntropia = time.time() #tempo de filtragem entropia
train_X, Val_X, train_y, Val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y) # divisao do conjunto de dados
train_X, train_y, Val_X, Val_y, test_X, test_y = carregarPeparar(train_X, train_y, Val_X, Val_y, test_X, test_y, 28, 1) #padronização e categorização

# Criaçaõ do Modelo
model = criacaoModeloFashion() # instanciando o modelo

# Treinamento
tempoIni = time.time() #tempo de filtragem entropia
history = model.fit(train_X, train_y, epochs = 10, batch_size = 128, validation_data=(Val_X, Val_y), verbose = 1)
tempoFim = time.time() #tempo de filtragem entropia
tempoTotal = round(tempoFim - tempoIni,2)
tempoTotalCalEntropia = round(tempoFimCalEntropia - tempoIniCalEntropia,2)

print("train_X: ",len(train_X))
print("train_y: ",len(train_y))
print("Val_X: ",len(Val_X))
print("Val_y: ",len(Val_y))
print("test_X: ",len(test_X))
print("test_y: ",len(test_y))
print("tempoTotal de treinamento: ", tempoTotal)
print("tempo Total CalEntropia: ", tempoTotalCalEntropia)

avaliacao_statistica(train_X, train_y, Val_X, Val_y, test_X, test_y)
plotAcuraciaLoss(history)

## [Fashion MNIST] Entropy Filter:
  * Filtered Dataset
  * Low Entropy in both training and validation data.
  * Cross-validation


In [ ]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from numpy import mean, std
from matplotlib import pyplot
from sklearn.model_selection import KFold
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.optimizers import SGD
import numpy as np
from scipy.stats import entropy
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
import time
import tensorflow as tf


def criacaoModeloFM1():
  model = Sequential()
  model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(28, 28, 1)))
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
  model.add(Dropout(rate=0.45))

  model.add(Flatten())
  model.add(Dense(units=128, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dropout(rate=0.25))
  model.add(Dense(units=10, activation='softmax'))

  opt = SGD(learning_rate=0.01, momentum =0.9)
  model.compile(optimizer=opt, loss = 'categorical_crossentropy', metrics= ['accuracy'])
  return model

def crossValidation(combinado_treino_X, combinado_validacao_y,n_folds = 5):
    #divide seus dados em k folds, e treina no k−1 e valida em 1 fold, então a proporção de dados usados para validação é 1/k.
    #2 folds (50% de validação): Treina em 50% dos dados e valida nos outros 50%.
    #5 folds (20% de validação): Treina em 80% dos dados e valida em 20%.
    #10 folds (10% de validação): Treina em 90% dos dados e valida em 10%.
    n_fold = 5 if n_folds == 0 else n_folds
    datagen = ImageDataGenerator(rotation_range=15,horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1)
    datagen.fit(combinado_treino_X)

    kf = KFold(n_splits = n_folds, shuffle = True, random_state = 42)  #divisão do conjunto de dados
    fold_no = 1
    for train_index, val_index in kf.split(train_X):
        train_data, val_data = train_X[train_index], train_X[val_index]
        train_label, val_label = train_y[train_index], train_y[val_index]

        print(f"Treinando a rede no folder numero: {fold_no}")
        model = criacaoModeloFM1() # instanciando o modelo
        inicio = time.time() #tempo de inicio
        history = model.fit(train_data, train_label, epochs = 10, batch_size = 128, verbose = 1, validation_data = (val_data, val_label))
        treinamentoF = time.time() #tempo final
        tempoTreino = round(treinamentoF - inicio,2)
        scores = model.evaluate(val_data, val_label, verbose = 1)
        print(f"Num do Folder: {fold_no}: Taxa de Loss: {scores[0]}, Taxa de Acuracia: {scores[1]*100:.2f}%, Tamanho do Conjunto de Dados: {len(train_data)}, Tempo de Treino: {tempoTreino}")
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy']) # Métrica de validação
        plt.title(f'Model Accuracy: {fold_no}')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss']) # Métrica de validação
        plt.title(f'Model Loss: {fold_no}')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.tight_layout()

        # Para o conjunto de TREINAMENTO
        print("Dados de Treino")
        predictions_train = model.predict(train_data) # Previsões
        classePredita_train = np.argmax(predictions_train, axis=1)
        classeVerdadeira_train = np.argmax(train_label, axis=1)
        cm_train = confusion_matrix(classeVerdadeira_train, classePredita_train)
        disp = ConfusionMatrixDisplay(cm_train)
        disp.plot()
        plt.title('Confusion Matrix - Training')
        plt.show()

        acc_train = accuracy_score(classeVerdadeira_train, classePredita_train) * 100  # Acurácia
        print("Accuracy (Treinamento): ", '%.3f' % (acc_train))
        sens_train = recall_score(classeVerdadeira_train, classePredita_train, average='macro')  # Sensibilidade
        print("Recall (Treinamento): ", '%.3f' % (sens_train))
        loss_train = history.history['loss'][-1]
        print("Loss (Treinamento): ", '%.3f' % loss_train)

        print("--------------------------------------------------")
        print("Dados de Validação")
        # Para o conjunto de VALIDAÇÃO
        predictions_val = model.predict(val_data) # Previsões
        classePredita_val = np.argmax(predictions_val, axis=1)
        classeVerdadeira_val = np.argmax(val_label, axis=1)
        cm_val = confusion_matrix(classeVerdadeira_val, classePredita_val)
        disp = ConfusionMatrixDisplay(cm_val)
        disp.plot()
        plt.title('Confusion Matrix - Validation')
        plt.show()

        acc_val = accuracy_score(classeVerdadeira_val, classePredita_val) * 100  # Acurácia
        print("Accuracy (Validação): ", '%.3f' % (acc_val))
        sens_val = recall_score(classeVerdadeira_val, classePredita_val, average='macro')  # Sensibilidade
        print("Recall (Validação): ", '%.3f' % (sens_val))
        loss_val = history.history['val_loss'][-1]
        print("Loss (Validação): ", '%.3f' % loss_val)

        fold_no += 1
        print("\n")

#carregar os dados
(train_X, train_y), (test_X, test_y) = tf.keras.datasets.fashion_mnist.load_data()
tempoEntropiaIni = time.time()
train_X, train_y, test_X, test_y = filtrar_entropia_median_train_teste(train_X, train_y, test_X, test_y,10) #novo filtro da reunião 03-10-2023
tempoEntropiaFim = time.time()
train_X = train_X.reshape((train_X.shape[0], 28, 28, 1))
test_X = test_X.reshape((test_X.shape[0], 28, 28, 1))
train_X = train_X.astype("float")/255.0
test_X = test_X.astype("float")/255.0
train_y = to_categorical(train_y, 10)
test_y = to_categorical(test_y, 10)

train_X = np.concatenate([train_X, test_X], axis=0)
train_y = np.concatenate([train_y, test_y], axis=0)

#criar/instanciar o modelo para validação cruzada
crossValidation(train_X,train_y, 5)


# CIFAR-10
* Dataset of 32x32 pixel color images, containing 60,000 images distributed into 10 classes, such as animals and vehicles, with 6,000 images per class.


## Creation and definition of all functions

1. Entropy Filter
2. Loading, Preprocessing, and Handling
3. Model Creation
4. Statistical Predictions
5. Accuracy and Loss Plot


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator


def entropia(pk, base=10, max=1):
    pk = pk / np.sum(pk)
    pk = pk[pk > 0]
    result = -np.sum(pk * np.log(pk) / np.log(base))
    return result

def filtrar_entropia_classe(train_X, train_y, totalClasses):  # filtro de entropia
    train_Xextend = []
    train_yextend = []
    for label in range(totalClasses):
        indices_originais = np.where(train_y == label)[0] #pega os indices originais da classe em label range
        indicesDasImagensDaClasse = train_X[indices_originais] #Os indices das imagens da classe são armazenados
        #Calculo a entropia para cada imagem da classe e em seguida enumera os indices da classe.
        tuplas = [(indices_originais[index], entropia(img)) for index, img in enumerate(indicesDasImagensDaClasse)]
        #Ordena as tuplas com base no segundo elemento, ou seja com base na entropia
        local_ordenado = sorted(tuplas, key=lambda x: x[1])
        #pega o tamanho da classe ordenada e separa a mediana da classe.
        n = len(local_ordenado)
        if n % 2 == 1:
            median = local_ordenado[n // 2][1]
        else:
            median = (local_ordenado[n // 2 - 1][1] + local_ordenado[n // 2][1]) / 2.0
        #Todas as imagens ordenadas pela entropia abaixo da medina são armezenadas nos indices_filtrados_da_Classe
        indices_filtrados_da_classe = [item[0] for item in local_ordenado if item[1] <= median]
        #as imagens abaixo da mediana são anexadas a cada nova interação em uma nova variavel.
        train_Xextend.extend([train_X[i] for i in indices_filtrados_da_classe])
        train_yextend.extend([train_y[i] for i in indices_filtrados_da_classe])
    return np.array(train_Xextend), np.array(train_yextend)

def filtrar_entropia_median_train_teste(train_X, train_y, test_X, test_y, totalClasses):  # filtro de entropia
    train_Xextend = []
    train_yextend = []
    for label in range(totalClasses):
        indices_originais = np.where(train_y == label)[0] #pega os indices originais da classe em label range
        indicesDasImagensDaClasse = train_X[indices_originais] #Os indices das imagens da classe são armazenados
        #Calculo a entropia para cada imagem da classe e em seguida enumera os indices da classe.
        tuplas = [(indices_originais[index], entropia(img)) for index, img in enumerate(indicesDasImagensDaClasse)]
        #Ordena as tuplas com base no segundo elemento, ou seja com base na entropia
        local_ordenado = sorted(tuplas, key=lambda x: x[1])
        #pega o tamanho da classe ordenada e separa a mediana da classe.
        n = len(local_ordenado)
        if n % 2 == 1:
            median = local_ordenado[n // 2][1]
        else:
            median = (local_ordenado[n // 2 - 1][1] + local_ordenado[n // 2][1]) / 2.0
        #Todas as imagens ordenadas pela entropia abaixo da medina são armezenadas nos indices_filtrados_da_Classe
        indices_filtrados_da_classe = [item[0] for item in local_ordenado if item[1] <= median]
        #as imagens abaixo da mediana são anexadas a cada nova interação em uma nova variavel.
        train_Xextend.extend([train_X[i] for i in indices_filtrados_da_classe])
        train_yextend.extend([train_y[i] for i in indices_filtrados_da_classe])
    test_Xextend = []
    test_yextend = []
    for label in range(totalClasses):
        indices_originais = np.where(test_y == label)[0] #pega os indices originais da classe em label range
        indicesDasImagensDaClasse = test_X[indices_originais] #Os indices das imagens da classe são armazenados
        #Calculo a entropia para cada imagem da classe e em seguida enumera os indices da classe.
        tuplas = [(indices_originais[index], entropia(img)) for index, img in enumerate(indicesDasImagensDaClasse)]
        #Ordena as tuplas com base no segundo elemento, ou seja com base na entropia
        local_ordenado = sorted(tuplas, key=lambda x: x[1])
        #pega o tamanho da classe ordenada e separa a mediana da classe.
        n = len(local_ordenado)
        if n % 2 == 1:
            median = local_ordenado[n // 2][1]
        else:
            median = (local_ordenado[n // 2 - 1][1] + local_ordenado[n // 2][1]) / 2.0
        #Todas as imagens ordenadas pela entropia abaixo da medina são armezenadas nos indices_filtrados_da_Classe
        indices_filtrados_da_classe = [item[0] for item in local_ordenado if item[1] <= median]
        #as imagens abaixo da mediana são anexadas a cada nova interação em uma nova variavel.
        test_Xextend.extend([test_X[i] for i in indices_filtrados_da_classe])
        test_yextend.extend([test_y[i] for i in indices_filtrados_da_classe])
    return np.array(train_Xextend), np.array(train_yextend), np.array(test_Xextend), np.array(test_yextend)

def carregarPeparar(train_X, train_y, Val_X, Val_y, test_X, test_y, d, c): # Carregamento e Tratamento
  train_X = train_X.reshape((train_X.shape[0], d, d, c))   # Ajuste da dimensão
  Val_X = Val_X.reshape((Val_X.shape[0], d, d, c))   # Ajuste da dimensão
  test_X = test_X.reshape((test_X.shape[0], d, d, c))    # Ajuste da dimensão
  train_X = train_X.astype("float")/255.0     # Normalização
  Val_X = Val_X.astype("float")/255.0     # Normalização
  test_X = test_X.astype("float")/255.0     # Normalização
  train_y = to_categorical(train_y, 10) #10 classes possiveis   -   # to Categorical para as classes
  Val_y = to_categorical(Val_y, 10) #10 classes possiveis   -   # to Categorical para as classes
  test_y = to_categorical(test_y, 10) #10 classes possiveis     -   # to Categorical para as classes
  return train_X, train_y, Val_X, Val_y, test_X, test_y

def carregarPepararValidacaoCruzada(train_X, train_y, test_X, test_y, d, c): # Carregamento e Tratamento
  train_X = train_X.reshape((train_X.shape[0], d, d, c))   # Ajuste da dimensão
  test_X = test_X.reshape((test_X.shape[0], d, d, c))    # Ajuste da dimensão
  train_X = train_X.astype("float")/255.0     # Normalização
  test_X = test_X.astype("float")/255.0     # Normalização
  train_y = to_categorical(train_y, 10) #10 classes possiveis   -   # to Categorical para as classes
  test_y = to_categorical(test_y, 10) #10 classes possiveis     -   # to Categorical para as classes
  return train_X, train_y, test_X, test_y

def avaliacao_statistica(train_X, train_y, Val_X, Val_y, test_X, test_y):
    # Para o conjunto de TREINAMENTO
    print("Dados de Treino")
    predictions_train = model.predict(train_X) # Previsões
    classePredita_train = np.argmax(predictions_train, axis=1)
    classeVerdadeira_train = np.argmax(train_y, axis=1)
    cm_train = confusion_matrix(classeVerdadeira_train, classePredita_train)

    disp = ConfusionMatrixDisplay(cm_train)
    disp.plot()
    plt.title('Confusion Matrix - Training')
    plt.show()

    acc_train = accuracy_score(classeVerdadeira_train, classePredita_train) * 100  # Acurácia
    print("Accuracy (Treinamento): ", '%.3f' % (acc_train))
    sens_train = recall_score(classeVerdadeira_train, classePredita_train, average='macro')  # Sensibilidade
    print("Recall (Treinamento): ", '%.3f' % (sens_train))
    loss_train = history.history['loss'][-1]
    print("Loss (Treinamento): ", '%.3f' % loss_train)

    print("--------------------------------------------------")
    print("Dados de Validação")
    # Para o conjunto de VALIDAÇÃO
    predictions_val = model.predict(Val_X) # Previsões
    classePredita_val = np.argmax(predictions_val, axis=1)
    classeVerdadeira_val = np.argmax(Val_y, axis=1)
    cm_val = confusion_matrix(classeVerdadeira_val, classePredita_val)

    disp = ConfusionMatrixDisplay(cm_val)
    disp.plot()
    plt.title('Confusion Matrix - Validation')
    plt.show()

    acc_val = accuracy_score(classeVerdadeira_val, classePredita_val) * 100  # Acurácia
    print("Accuracy (Validação): ", '%.3f' % (acc_val))
    sens_val = recall_score(classeVerdadeira_val, classePredita_val, average='macro')  # Sensibilidade
    print("Recall (Validação): ", '%.3f' % (sens_val))
    loss_val = history.history['val_loss'][-1]
    print("Loss (Validação): ", '%.3f' % loss_val)

    print("--------------------------------------------------")
    print("Dados de Teste")
    # Para o conjunto de TESTE
    predictions_test = model.predict(test_X) # Previsões
    classePredita_test = np.argmax(predictions_test, axis=1)
    classeVerdadeira_test = np.argmax(test_y, axis=1)
    cm_test = confusion_matrix(classeVerdadeira_test, classePredita_test)

    disp = ConfusionMatrixDisplay(cm_test)
    disp.plot()
    plt.title('Confusion Matrix - Test')
    plt.show()

    acc_test = accuracy_score(classeVerdadeira_test, classePredita_test) * 100  # Acurácia
    print("Accuracy (Teste): ", '%.3f' % (acc_test))
    sens_test = recall_score(classeVerdadeira_test, classePredita_test, average='macro')  # Sensibilidade
    print("Recall (Teste): ", '%.3f' % (sens_test))
    loss_test = history.history['val_loss'][-1]
    print("Loss (Teste): ", '%.3f' % loss_test)

def plotAcuraciaLoss(history): #plots
  plt.figure(figsize=(12, 5))
  plt.subplot(1, 2, 1)
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('Model Accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  plt.subplot(1, 2, 2)
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model Loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  plt.tight_layout()

def plotarDadosTrain(index_inicio, qtd, train_X): # Exemplo: apartir do indice N, exibe K imagens do conjunto de dados, D
  base = 330
  import matplotlib.pyplot as plt
  for i in range(index_inicio, index_inicio + qtd):
      plt.subplot(base + 1 + (i - index_inicio))
      plt.imshow(train_X[i], cmap=plt.get_cmap('gray'))
  plt.show()
  print(train_y[index_inicio:index_inicio + qtd])

def entHistogram(testar, num):
  import matplotlib.pyplot as plt
  if num == 1:
    title = 'Class histogram train_y'
  elif num == 2:
    title = 'Class histogram Val_y'
  elif num == 3:
    title = 'Class histogram test_y'
  plt.hist(testar, bins=10, edgecolor='black', alpha=0.7)
  plt.title(title)
  plt.xlabel('Value')
  plt.ylabel('Frequency')
  plt.show()

def tempDecorrido(inicio, importacaoF, padrozinacaoF, criacaoModeloF, treinamentoF, entropiaF = None): # x = time.time() || inicio, importacaoF, padrozinacaoF, criacaoModeloF, treinamentoF, entropiaF
  elapsed_time = [["Carregar DataSet", 0], ["Filtro Entropia", 0], ["Pre-processamento", 0], ["Criação do Modelo", 0], ["Treinamento", 0], ["Inicio à Fim Execução" , 0]]
  elapsed_time[0][1] = importacaoF - inicio  # tempo de inicio de execução até o final da importação do dataset
  if (entropiaF != None): elapsed_time[1][1] = entropiaF - importacaoF  # Tempo gasto Inicio é importaçãoF - Execução da Entropia
  elapsed_time[2][1] = padrozinacaoF - importacaoF  # pega o tempo da entropia e subtrai do tempo apos a padronização para verificar quanto tempo padronizacao demorou
  elapsed_time[3][1] = criacaoModeloF - padrozinacaoF  # tempo da padronizacao - tempo apos a criacao do modelo para verificar tempo decorido
  elapsed_time[4][1] = treinamentoF - criacaoModeloF  # tempo inicial é marcado pela criacaoModeloF - tempoTreinamento que marca o momento que treinamento terminou
  elapsed_time[5][1] = treinamentoF - inicio #tempo total de execução inicio| importações até final da Execução | treinamento
  for index, tempo in enumerate(elapsed_time):
    if entropiaF is None and index == 1:
      continue
    print("{}: {:.4f}".format(tempo[0], round(tempo[1], 2)) + str(" em milissegundos"))

def criacaoModeloG9():
    opt=Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='elu', input_shape=(32, 32, 3), padding='same'))
    model.add(BatchNormalization(axis=-1))
    model.add(Conv2D(32, (3, 3), activation='elu', padding='same'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), activation='elu', padding='same'))
    model.add(BatchNormalization(axis=-1))
    model.add(Conv2D(64, (3, 3), activation='elu', padding='same'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (3, 3), activation='elu', padding='same'))
    model.add(BatchNormalization(axis=-1))
    model.add(Conv2D(128, (3, 3), activation='elu', padding='same'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(512, activation='elu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer=opt)
    return model

## [CIFAR-10] Full Dataset Training:
  * Without entropy selection
  * Full Dataset


In [ ]:
(train_X, train_y), (test_X, test_y) = cifar10.load_data() # Carregar o conjunto de dados # Importar e Treinar
train_X, Val_X, train_y, Val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y) # divisao do conjunto de dados
entHistogram(train_y, 1)
entHistogram(Val_y, 2)
entHistogram(test_y, 3)
print(len(train_X))
print(len(train_y))
print(len(test_X))
print(len(test_y))

In [ ]:
(train_X, train_y), (test_X, test_y) = cifar10.load_data() # Carregar o conjunto de dados # Importar e Treinar
train_X, Val_X, train_y, Val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y) # divisao do conjunto de dados
train_X, train_y, Val_X, Val_y, test_X, test_y = carregarPeparar(train_X, train_y, Val_X, Val_y, test_X, test_y, 32, 3) #padronização e categorização
datagen = ImageDataGenerator(rotation_range=15,horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1)
datagen.fit(train_X)

model = criacaoModeloG9()
inicio = time.time() #tempo de inicio
history=model.fit(datagen.flow(train_X, train_y, batch_size=128),
                    steps_per_epoch = len(train_X) / 128, epochs=50, validation_data=(Val_X, Val_y))
treinamentoF = time.time() #tempo final
tempoTreino = round(treinamentoF - inicio,2)
print("Tempo de treinamento: ", tempoTreino)
print("train_X: ",len(train_X))
print("train_y: ",len(train_y))
print("Val_X: ",len(Val_X))
print("Val_y: ",len(Val_y))
print("test_X: ",len(test_X))
print("test_y: ",len(test_y))
avaliacao_statistica(train_X, train_y, Val_X, Val_y, test_X, test_y)
plotAcuraciaLoss(history)

## [CIFAR-10] Random and Stratified Training:
  * Without entropy selection
  * Stratified dataset divided proportionally in half


In [ ]:
(train_X, train_y), (test_X, test_y) = cifar10.load_data() # Carregar o conjunto de dados # Importar e Treinar
partial_train_X, _, partial_train_y, _ = train_test_split(train_X, train_y, test_size=0.5, random_state=42, stratify=train_y)
train_X, Val_X, train_y, Val_y = train_test_split(partial_train_X, partial_train_y, test_size=(0.2), random_state=42, stratify=partial_train_y)
entHistogram(train_y, 1)
entHistogram(Val_y, 2)
entHistogram(test_y, 3)
print(len(train_X))
print(len(train_y))
print(len(test_X))
print(len(test_y))

In [ ]:
(train_X, train_y), (test_X, test_y) = cifar10.load_data() # Carregar o conjunto de dados # Importar e Treinar
partial_train_X, _, partial_train_y, _ = train_test_split(train_X, train_y, test_size=0.5, random_state=42, stratify=train_y)
train_X, Val_X, train_y, Val_y = train_test_split(partial_train_X, partial_train_y, test_size=(0.2), random_state=42, stratify=partial_train_y)
train_X, train_y, Val_X, Val_y, test_X, test_y = carregarPeparar(train_X, train_y, Val_X, Val_y, test_X, test_y, 32, 3) #padronização e categorização
datagen = ImageDataGenerator(rotation_range=15,horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1)
datagen.fit(train_X)

model = criacaoModeloG9()
inicio = time.time() #tempo de inicio
history=model.fit(datagen.flow(train_X, train_y, batch_size=128),
                    steps_per_epoch = len(train_X) / 128, epochs=50, validation_data=(Val_X, Val_y))
treinamentoF = time.time() #tempo final
tempoTreino = round(treinamentoF - inicio,2)
print("Tempo de treinamento: ", tempoTreino)
print("train_X: ",len(train_X))
print("train_y: ",len(train_y))
print("Val_X: ",len(Val_X))
print("Val_y: ",len(Val_y))
print("test_X: ",len(test_X))
print("test_y: ",len(test_y))
avaliacao_statistica(train_X, train_y, Val_X, Val_y, test_X, test_y)
plotAcuraciaLoss(history)

## [CIFAR-10] Entropy Filter:
  * Filtered Dataset
  * Low Entropy in training data causing "noise" in validation.


In [ ]:
(train_X, train_y), (test_X, test_y) = cifar10.load_data() # Carregar o conjunto de dados # Importar e Treinar
train_X, Val_X, train_y, Val_y = train_test_split(train_X, train_y, test_size=0.5, random_state=42, stratify=train_y) # divisao do conjunto de dados
train_X, train_y = filtrar_entropia_classe(train_X, train_y,10)
entHistogram(train_y, 1)
entHistogram(Val_y, 2)
entHistogram(test_y, 3)
print(len(train_X))
print(len(train_y))
print(len(test_X))
print(len(test_y))

In [ ]:
(train_X, train_y), (test_X, test_y) = cifar10.load_data() # Carregar o conjunto de dados # Importar e Treinar
train_X, Val_X, train_y, Val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y) # divisao do conjunto de dados

tempoCalEntropiaIni = time.time()
train_X, train_y = filtrar_entropia_classe(train_X, train_y,10)
tempoCalEntropiaFim = time.time()

train_X, train_y, Val_X, Val_y, test_X, test_y = carregarPeparar(train_X, train_y, Val_X, Val_y, test_X, test_y, 32, 3) #padronização e categorização
datagen = ImageDataGenerator(rotation_range=15,horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1)
datagen.fit(train_X)
model = criacaoModeloG9()
inicio = time.time() #tempo de inicio
history=model.fit(datagen.flow(train_X, train_y, batch_size=128),
                    steps_per_epoch = len(train_X) / 128, epochs=50, validation_data=(Val_X, Val_y))
treinamentoF = time.time() #tempo final
tempoTreino = round(treinamentoF - inicio,2)
print("Tempo de treinamento: ", tempoTreino)
tempoCalEntropia = round(tempoCalEntropiaFim - tempoCalEntropiaIni,2)
print("Tempo de calculo da entropia: ", tempoCalEntropia)
print("train_X: ",len(train_X))
print("train_y: ",len(train_y))
print("Val_X: ",len(Val_X))
print("Val_y: ",len(Val_y))
print("test_X: ",len(test_X))
print("test_y: ",len(test_y))
avaliacao_statistica(train_X, train_y, Val_X, Val_y, test_X, test_y)
plotAcuraciaLoss(history)

## [CIFAR-10] Entropy Filter:
  * Filtered Dataset
  * Low Entropy in both training and validation data.


In [ ]:
(train_X, train_y), (test_X, test_y) = cifar10.load_data() # Carregar o conjunto de dados # Importar e Treinar
train_X, train_y = filtrar_entropia_classe(train_X, train_y,10)
train_X, Val_X, train_y, Val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y) # divisao do conjunto de dados
entHistogram(train_y, 1)
entHistogram(Val_y, 2)
entHistogram(test_y, 3)
print(len(train_X))
print(len(train_y))
print(len(test_X))
print(len(test_y))

In [ ]:
(train_X, train_y), (test_X, test_y) = cifar10.load_data() # Carregar o conjunto de dados # Importar e Treinar
tempoCalEntropiaIni = time.time()
train_X, train_y = filtrar_entropia_classe(train_X, train_y,10)
tempoCalEntropiaFim = time.time()

train_X, Val_X, train_y, Val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y) # divisao do conjunto de dados
train_X, train_y, Val_X, Val_y, test_X, test_y = carregarPeparar(train_X, train_y, Val_X, Val_y, test_X, test_y, 32, 3) #padronização e categorização
datagen = ImageDataGenerator(rotation_range=15,horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1)
datagen.fit(train_X)
model = criacaoModeloG9()
inicio = time.time() #tempo de inicio
history=model.fit(datagen.flow(train_X, train_y, batch_size=128),
                    steps_per_epoch = len(train_X) / 128, epochs=50, validation_data=(Val_X, Val_y))
treinamentoF = time.time() #tempo final
tempoTreino = round(treinamentoF - inicio,2)
print("Tempo de treinamento: ", tempoTreino)
tempoCalEntropia = round(tempoCalEntropiaFim - tempoCalEntropiaIni,2)
print("Tempo de calculo da entropia: ", tempoCalEntropia)
print("train_X: ",len(train_X))
print("train_y: ",len(train_y))
print("Val_X: ",len(Val_X))
print("Val_y: ",len(Val_y))
print("test_X: ",len(test_X))
print("test_y: ",len(test_y))
avaliacao_statistica(train_X, train_y, Val_X, Val_y, test_X, test_y)
plotAcuraciaLoss(history)

## [CIFAR-10] Entropy Filter:
  * Filtered Dataset
  * Low Entropy in both training and validation data.
  * Cross-validation


In [ ]:
#Corrigindo o metodo que estava errado:


from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def crossValidation(train_X,train_y,n_folds = 5):
    #divide seus dados em k folds, e treina no k−1 e valida em 1 fold, então a proporção de dados usados para validação é 1/k.
    #2 folds (50% de validação): Treina em 50% dos dados e valida nos outros 50%.
    #5 folds (20% de validação): Treina em 80% dos dados e valida em 20%.
    #10 folds (10% de validação): Treina em 90% dos dados e valida em 10%.
    n_fold = 5 if n_folds == 0 else n_folds
    datagen = ImageDataGenerator(rotation_range=15,horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1)
    datagen.fit(train_X)

    kf = KFold(n_splits = n_folds, shuffle = True, random_state = 42)  #divisão do conjunto de dados
    fold_no = 1
    for train_index, val_index in kf.split(train_X):
        train_data, val_data = train_X[train_index], train_X[val_index]
        train_label, val_label = train_y[train_index], train_y[val_index]

        print(f"Treinando a rede no folder numero: {fold_no}")
        model = criacaoModeloG9() # instanciando o modelo
        inicio = time.time() #tempo de inicio
        history=model.fit(datagen.flow(train_data, train_label, batch_size=128),
                          steps_per_epoch = len(train_data) / 128, epochs=50, validation_data=(val_data, val_label))

        treinamentoF = time.time() #tempo final
        tempoTreino = round(treinamentoF - inicio,2)
        scores = model.evaluate(val_data, val_label, verbose = 1)


        print(f"Num do Folder: {fold_no}: Taxa de Loss: {scores[0]}, Taxa de Acuracia: {scores[1]*100:.2f}%, Tamanho do Conjunto de Dados: {len(train_data)}, Tempo de Treino: {tempoTreino}")
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title(f'Model Accuracy: {fold_no}')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title(f'Model Loss: {fold_no}')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.tight_layout()

        # Para o conjunto de TREINAMENTO
        print("Dados de Treino")
        predictions_train = model.predict(train_data) # Previsões
        classePredita_train = np.argmax(predictions_train, axis=1)
        classeVerdadeira_train = np.argmax(train_label, axis=1)
        cm_train = confusion_matrix(classeVerdadeira_train, classePredita_train)
        disp = ConfusionMatrixDisplay(cm_train)
        disp.plot()
        plt.title('Confusion Matrix - Training')
        plt.show()

        acc_train = accuracy_score(classeVerdadeira_train, classePredita_train) * 100  # Acurácia
        print("Accuracy (Treinamento): ", '%.3f' % (acc_train))
        sens_train = recall_score(classeVerdadeira_train, classePredita_train, average='macro')  # Sensibilidade
        print("Recall (Treinamento): ", '%.3f' % (sens_train))
        loss_train = history.history['loss'][-1]
        print("Loss (Treinamento): ", '%.3f' % loss_train)

        print("--------------------------------------------------")
        print("Dados de Validação")
        # Para o conjunto de VALIDAÇÃO
        predictions_val = model.predict(val_data) # Previsões
        classePredita_val = np.argmax(predictions_val, axis=1)
        classeVerdadeira_val = np.argmax(val_label, axis=1)
        cm_val = confusion_matrix(classeVerdadeira_val, classePredita_val)
        disp = ConfusionMatrixDisplay(cm_val)
        disp.plot()
        plt.title('Confusion Matrix - Validation')
        plt.show()

        acc_val = accuracy_score(classeVerdadeira_val, classePredita_val) * 100  # Acurácia
        print("Accuracy (Validação): ", '%.3f' % (acc_val))
        sens_val = recall_score(classeVerdadeira_val, classePredita_val, average='macro')  # Sensibilidade
        print("Recall (Validação): ", '%.3f' % (sens_val))
        loss_val = history.history['val_loss'][-1]
        print("Loss (Validação): ", '%.3f' % loss_val)

        fold_no += 1
        print("\n")

#carregar os dados
from keras.datasets import cifar10
(train_X, train_y), (test_X, test_y) = cifar10.load_data() # Carregar o conjunto de dados # Importar e Treinar
train_X, train_y, test_X, test_y = filtrar_entropia_median_train_teste(train_X, train_y, test_X, test_y,10) #novo filtro da reunião 03-10-2023
train_X, train_y, test_X, test_y = carregarPepararValidacaoCruzada(train_X, train_y, test_X, test_y, 32, 3) #padronização e categorização

train_X = np.concatenate([train_X, test_X], axis=0)
train_y = np.concatenate([train_y, test_y], axis=0)

#criar/instanciar o modelo para validação cruzada
crossValidation(train_X,train_y, 5)

# CIFAR-100
* Similar to CIFAR-10, but with 100 classes containing 600 images each. The classes are grouped into 20 superclasses, providing a greater diversity of objects and beings.


### [CIFAR100] Creation and definition of all main functions

1. Entropy Filter
2. Loading, Preprocessing, and Handling
3. Model Creation
4. Statistical Predictions
5. Accuracy and Loss Plot


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from datetime import datetime
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, UpSampling2D, Dropout, BatchNormalization, GlobalAveragePooling2D
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import cifar100
from keras.datasets import mnist
from scipy.stats import entropy
from keras.layers import MaxPooling2D
from keras.optimizers import SGD
from sklearn import metrics
from sklearn.metrics import recall_score
from PIL import Image
import time
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.resnet50 import ResNet50

def entropia(pk, base=10, max=1):
    pk = pk / np.sum(pk)
    pk = pk[pk > 0]
    result = -np.sum(pk * np.log(pk) / np.log(base))
    return result

def filtrar_entropia_classe(train_X, train_y, totalClasses):  # filtro de entropia
    train_Xextend = []
    train_yextend = []
    for label in range(totalClasses):
        indices_originais = np.where(train_y == label)[0] #pega os indices originais da classe em label range
        indicesDasImagensDaClasse = train_X[indices_originais] #Os indices das imagens da classe são armazenados
        #Calculo a entropia para cada imagem da classe e em seguida enumera os indices da classe.
        tuplas = [(indices_originais[index], entropia(img)) for index, img in enumerate(indicesDasImagensDaClasse)]
        #Ordena as tuplas com base no segundo elemento, ou seja com base na entropia
        local_ordenado = sorted(tuplas, key=lambda x: x[1])
        #pega o tamanho da classe ordenada e separa a mediana da classe.
        n = len(local_ordenado)
        if n % 2 == 1:
            median = local_ordenado[n // 2][1]
        else:
            median = (local_ordenado[n // 2 - 1][1] + local_ordenado[n // 2][1]) / 2.0
        #Todas as imagens ordenadas pela entropia abaixo da medina são armezenadas nos indices_filtrados_da_Classe
        indices_filtrados_da_classe = [item[0] for item in local_ordenado if item[1] <= median]
        #as imagens abaixo da mediana são anexadas a cada nova interação em uma nova variavel.
        train_Xextend.extend([train_X[i] for i in indices_filtrados_da_classe])
        train_yextend.extend([train_y[i] for i in indices_filtrados_da_classe])
    return np.array(train_Xextend), np.array(train_yextend)

def filtrar_entropia_median_train_teste(train_X, train_y, test_X, test_y, totalClasses):  # filtro de entropia
    train_Xextend = []
    train_yextend = []
    for label in range(totalClasses):
        indices_originais = np.where(train_y == label)[0] #pega os indices originais da classe em label range
        indicesDasImagensDaClasse = train_X[indices_originais] #Os indices das imagens da classe são armazenados
        #Calculo a entropia para cada imagem da classe e em seguida enumera os indices da classe.
        tuplas = [(indices_originais[index], entropia(img)) for index, img in enumerate(indicesDasImagensDaClasse)]
        #Ordena as tuplas com base no segundo elemento, ou seja com base na entropia
        local_ordenado = sorted(tuplas, key=lambda x: x[1])
        #pega o tamanho da classe ordenada e separa a mediana da classe.
        n = len(local_ordenado)
        if n % 2 == 1:
            median = local_ordenado[n // 2][1]
        else:
            median = (local_ordenado[n // 2 - 1][1] + local_ordenado[n // 2][1]) / 2.0
        #Todas as imagens ordenadas pela entropia abaixo da medina são armezenadas nos indices_filtrados_da_Classe
        indices_filtrados_da_classe = [item[0] for item in local_ordenado if item[1] <= median]
        #as imagens abaixo da mediana são anexadas a cada nova interação em uma nova variavel.
        train_Xextend.extend([train_X[i] for i in indices_filtrados_da_classe])
        train_yextend.extend([train_y[i] for i in indices_filtrados_da_classe])
    test_Xextend = []
    test_yextend = []
    for label in range(totalClasses):
        indices_originais = np.where(test_y == label)[0] #pega os indices originais da classe em label range
        indicesDasImagensDaClasse = test_X[indices_originais] #Os indices das imagens da classe são armazenados
        #Calculo a entropia para cada imagem da classe e em seguida enumera os indices da classe.
        tuplas = [(indices_originais[index], entropia(img)) for index, img in enumerate(indicesDasImagensDaClasse)]
        #Ordena as tuplas com base no segundo elemento, ou seja com base na entropia
        local_ordenado = sorted(tuplas, key=lambda x: x[1])
        #pega o tamanho da classe ordenada e separa a mediana da classe.
        n = len(local_ordenado)
        if n % 2 == 1:
            median = local_ordenado[n // 2][1]
        else:
            median = (local_ordenado[n // 2 - 1][1] + local_ordenado[n // 2][1]) / 2.0
        #Todas as imagens ordenadas pela entropia abaixo da medina são armezenadas nos indices_filtrados_da_Classe
        indices_filtrados_da_classe = [item[0] for item in local_ordenado if item[1] <= median]
        #as imagens abaixo da mediana são anexadas a cada nova interação em uma nova variavel.
        test_Xextend.extend([test_X[i] for i in indices_filtrados_da_classe])
        test_yextend.extend([test_y[i] for i in indices_filtrados_da_classe])
    return np.array(train_Xextend), np.array(train_yextend), np.array(test_Xextend), np.array(test_yextend)

def avaliacao_statistica(train_X, train_y, Val_X, Val_y, test_X, test_y):
    # Para o conjunto de TREINAMENTO
    print("Dados de Treino")
    predictions_train = model.predict(train_X) # Previsões
    classePredita_train = np.argmax(predictions_train, axis=1)
    classeVerdadeira_train = np.argmax(train_y, axis=1)
    cm_train = confusion_matrix(classeVerdadeira_train, classePredita_train)

    plt.figure(figsize=(32, 32))  # Define o tamanho da figura
    disp = ConfusionMatrixDisplay(cm_train)
    disp.plot()
    plt.title('Confusion Matrix - Training')
    plt.show()

    acc_train = accuracy_score(classeVerdadeira_train, classePredita_train) * 100  # Acurácia
    print("Accuracy (Treinamento): ", '%.3f' % (acc_train))
    sens_train = recall_score(classeVerdadeira_train, classePredita_train, average='macro')  # Sensibilidade
    print("Recall (Treinamento): ", '%.3f' % (sens_train))
    loss_train = history.history['loss'][-1]
    print("Loss (Treinamento): ", '%.3f' % loss_train)

    print("--------------------------------------------------")
    print("Dados de Validação")
    # Para o conjunto de VALIDAÇÃO
    predictions_val = model.predict(Val_X) # Previsões
    classePredita_val = np.argmax(predictions_val, axis=1)
    classeVerdadeira_val = np.argmax(Val_y, axis=1)
    cm_val = confusion_matrix(classeVerdadeira_val, classePredita_val)

    plt.figure(figsize=(32, 32))  # Define o tamanho da figura
    disp = ConfusionMatrixDisplay(cm_val)
    disp.plot()
    plt.title('Confusion Matrix - Validation')
    plt.show()

    acc_val = accuracy_score(classeVerdadeira_val, classePredita_val) * 100  # Acurácia
    print("Accuracy (Validação): ", '%.3f' % (acc_val))
    sens_val = recall_score(classeVerdadeira_val, classePredita_val, average='macro')  # Sensibilidade
    print("Recall (Validação): ", '%.3f' % (sens_val))
    loss_val = history.history['val_loss'][-1]
    print("Loss (Validação): ", '%.3f' % loss_val)

    print("--------------------------------------------------")
    print("Dados de Teste")
    # Para o conjunto de TESTE
    predictions_test = model.predict(test_X) # Previsões
    classePredita_test = np.argmax(predictions_test, axis=1)
    classeVerdadeira_test = np.argmax(test_y, axis=1)
    cm_test = confusion_matrix(classeVerdadeira_test, classePredita_test)

    plt.figure(figsize=(32, 32))  # Define o tamanho da figura
    disp = ConfusionMatrixDisplay(cm_test)
    disp.plot()
    plt.title('Confusion Matrix - Test')
    plt.show()

    acc_test = accuracy_score(classeVerdadeira_test, classePredita_test) * 100  # Acurácia
    print("Accuracy (Teste): ", '%.3f' % (acc_test))
    sens_test = recall_score(classeVerdadeira_test, classePredita_test, average='macro')  # Sensibilidade
    print("Recall (Teste): ", '%.3f' % (sens_test))
    loss_test = history.history['val_loss'][-1]
    print("Loss (Teste): ", '%.3f' % loss_test)


def plotAcuraciaLoss(history): #plots
  plt.figure(figsize=(12, 5))
  plt.subplot(1, 2, 1)
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('Model Accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  plt.subplot(1, 2, 2)
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model Loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  plt.tight_layout()

def entHistogram(testar, num):
  import matplotlib.pyplot as plt
  if num == 1:
    title = 'Class histogram train_y'
  elif num == 2:
    title = 'Class histogram Val_y'
  elif num == 3:
    title = 'Class histogram test_y'
  plt.hist(testar, bins=10, edgecolor='black', alpha=0.7)
  plt.title(title)
  plt.xlabel('Value')
  plt.ylabel('Frequency')
  plt.show()

def carregarPepararCifar100(train_X, train_y, Val_X, Val_y, test_X, test_y): # Carregamento e Tratamento
  train_X = train_X.astype("float")/255
  Val_X = Val_X.astype("float")/255
  test_X = test_X.astype("float")/255
  train_y = to_categorical(train_y, num_classes=100)
  Val_y = to_categorical(Val_y, num_classes=100)
  test_y = to_categorical(test_y, num_classes=100)
  return train_X, train_y, Val_X, Val_y, test_X, test_y

def carregarPepararValidacaoCruzadaCifar100(train_X, train_y, test_X, test_y): # Carregamento e Tratamento
  train_X = train_X.astype("float")/255
  test_X = test_X.astype("float")/255
  train_y = to_categorical(train_y, num_classes=100)
  test_y = to_categorical(test_y, num_classes=100)
  return train_X, train_y, test_X, test_y

def criarModeloCifar100():
  #Importing the Resnet Model
  from tensorflow.keras.applications.resnet50 import ResNet50
  resnet_model = ResNet50(
      include_top = False,
      weights = 'imagenet',
      input_shape = (224,224,3)
  )
  for layer in resnet_model.layers:
      if isinstance(layer, BatchNormalization):
          layer.trainable = True
      else:
          layer.trainable = False


  #reducao da taxa de aprendizagem
  from keras.callbacks import ReduceLROnPlateau
  learning_rate_reduction = ReduceLROnPlateau(
      monitor='val_accuracy',
      patience=2,
      verbose=1,
      factor=0.96,
      min_lr=1e-8)

  model = Sequential()
  model.add(UpSampling2D(size=(7, 7),interpolation='bilinear'))
  model.add(resnet_model)
  model.add(GlobalAveragePooling2D())
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(512, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=None))
  model.add(BatchNormalization())
  model.add(Dropout(0.25))
  model.add(Dense(100, activation='softmax'))
  optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3, momentum=0.9)
  model.compile(
      optimizer = optimizer,
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )
  return model, learning_rate_reduction


import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd

def return_label(preds):
    return np.argmax(preds, axis=1)

def avaliacao_statisticaCifar100(train_X, train_y, Val_X, Val_y, test_X, test_y, model, history):

    datasets = {
        "Training": (train_X, train_y),
        "Validation": (Val_X, Val_y),
        "Test": (test_X, test_y)
    }

    for name, data in datasets.items():
        X, y = data
        print(f"--------------------------------------------------")
        print(f"Dados de {name}")

        # Previsões
        predictions = model.predict(X)
        pred_labels = return_label(predictions)
        true_labels = return_label(y)

        # Preparando a matriz de confusão
        conf_mat = confusion_matrix(true_labels, pred_labels)

        # Exibindo a matriz de confusão usando heatmap
        df_cm = pd.DataFrame(conf_mat, index=[i for i in range(100)], columns=[i for i in range(100)])
        plt.figure(figsize=(32, 32))
        #sn.heatmap(df_cm, annot=True, fmt="d")
        sn.heatmap(df_cm, annot=True, fmt="d", cmap="viridis")
        plt.title(f'Confusion Matrix - {name}')
        plt.show()

        # Métricas
        acc = accuracy_score(true_labels, pred_labels) * 100
        sens = recall_score(true_labels, pred_labels, average='macro')

        if name == "Treinamento":
            loss = history.history['loss'][-1]
        else:
            loss = history.history['val_loss'][-1]

        print(f"Accuracy ({name}): ", '%.3f' % (acc))
        print(f"Recall ({name}): ", '%.3f' % (sens))
        print(f"Loss ({name}): ", '%.3f' % loss)

### [CIFAR100] Full Dataset Training:
  * Without entropy selection
  * Full Dataset


In [ ]:
(train_X, train_y), (test_X, test_y) = cifar100.load_data() # Carregar o conjunto de dados # Importar e Treinar
train_X, Val_X, train_y, Val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y) # divisao do conjunto de dados
train_X, train_y, Val_X, Val_y, test_X, test_y = carregarPepararCifar100(train_X, train_y, Val_X, Val_y, test_X, test_y)
datagen = ImageDataGenerator(rotation_range=15,horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1)
datagen.fit(train_X)

model, learning_rate_reduction = criarModeloCifar100()

tempoTreinamento = time.time() #tempo para treinamento
history = model.fit(
    datagen.flow(train_X, train_y, batch_size=128),
    steps_per_epoch=len(train_X) / 128,
    epochs=50,
    validation_data=(Val_X, Val_y),
    callbacks=[learning_rate_reduction]
)
tempoFimTreino = time.time() #tempo para treinamento
tempoFinalTreino = round(tempoFimTreino - tempoTreinamento,2)
print("Tempo de treinamento(s): ", tempoFinalTreino)
print("train_X: ",len(train_X))
print("train_y: ",len(train_y))
print("Val_X: ",len(Val_X))
print("Val_y: ",len(Val_y))
print("test_X: ",len(test_X))
print("test_y: ",len(test_y))
avaliacao_statisticaCifar100(train_X, train_y, Val_X, Val_y, test_X, test_y, model, history)
plotAcuraciaLoss(history)

### [CIFAR100] Random and Stratified Training:
  * Without entropy selection
  * Stratified dataset divided proportionally in half


In [ ]:
(train_X, train_y), (test_X, test_y) = cifar100.load_data() # Carregar o conjunto de dados # Importar e Treinar
partial_train_X, _, partial_train_y, _ = train_test_split(train_X, train_y, test_size=0.5, random_state=42, stratify=train_y)
train_X, Val_X, train_y, Val_y = train_test_split(partial_train_X, partial_train_y, test_size=(0.2), random_state=42, stratify=partial_train_y)
train_X, train_y, Val_X, Val_y, test_X, test_y = carregarPepararCifar100(train_X, train_y, Val_X, Val_y, test_X, test_y)
datagen = ImageDataGenerator(rotation_range=15,horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1)
datagen.fit(train_X)

model, learning_rate_reduction = criarModeloCifar100()

tempoTreinamento = time.time() #tempo para treinamento
history = model.fit(
    datagen.flow(train_X, train_y, batch_size=128),
    steps_per_epoch=len(train_X) / 128,
    epochs=50,
    validation_data=(Val_X, Val_y),
    callbacks=[learning_rate_reduction]
)
tempoFimTreino = time.time() #tempo para treinamento
tempoFinalTreino = round(tempoFimTreino - tempoTreinamento,2)
print("Tempo de treinamento(s): ", tempoFinalTreino)
print("train_X: ",len(train_X))
print("train_y: ",len(train_y))
print("Val_X: ",len(Val_X))
print("Val_y: ",len(Val_y))
print("test_X: ",len(test_X))
print("test_y: ",len(test_y))
avaliacao_statisticaCifar100(train_X, train_y, Val_X, Val_y, test_X, test_y, model, history)
plotAcuraciaLoss(history)

### [CIFAR100] Entropy Filter:
  * Filtered Dataset
  * Low Entropy in training data causing "noise" in validation.


In [ ]:
(train_X, train_y), (test_X, test_y) = cifar100.load_data() # Carregar o conjunto de dados # Importar e Treinar
train_X, Val_X, train_y, Val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y) # divisao do conjunto de dados
train_X, train_y = filtrar_entropia_classe(train_X, train_y,100)
entHistogram(train_y, 1)
entHistogram(Val_y, 2)
entHistogram(test_y, 3)
print(len(train_X))
print(len(train_y))
print(len(test_X))
print(len(test_y))

In [ ]:
(train_X, train_y), (test_X, test_y) = cifar100.load_data() # Carregar o conjunto de dados # Importar e Treinar
train_X, Val_X, train_y, Val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y) # divisao do conjunto de dados
tempoFiltroEntropiaInicio = time.time() #tempo para calculo do filtro
train_X, train_y = filtrar_entropia_classe(train_X, train_y,100)
tempoFiltroEntropiaFim = time.time() #Tempo final do calculo do filtro
train_X, train_y, Val_X, Val_y, test_X, test_y = carregarPepararCifar100(train_X, train_y, Val_X, Val_y, test_X, test_y)
datagen = ImageDataGenerator(rotation_range=15,horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1)
datagen.fit(train_X)

model, learning_rate_reduction = criarModeloCifar100()

tempoTreinamento = time.time() #tempo para treinamento
history = model.fit(
    datagen.flow(train_X, train_y, batch_size=128),
    steps_per_epoch=len(train_X) / 128,
    epochs=50,
    validation_data=(Val_X, Val_y),
    callbacks=[learning_rate_reduction]
)
tempoFimTreino = time.time() #tempo para treinamento
tempoFinalTreino = round(tempoFimTreino - tempoTreinamento,2)
print("Tempo de treinamento(s): ", tempoFinalTreino)
TempoCalEntropia = round(tempoFiltroEntropiaFim - tempoFiltroEntropiaInicio,2)
print("Tempo de Calculo Entropia(s): ", TempoCalEntropia)
print("train_X: ",len(train_X))
print("train_y: ",len(train_y))
print("Val_X: ",len(Val_X))
print("Val_y: ",len(Val_y))
print("test_X: ",len(test_X))
print("test_y: ",len(test_y))
avaliacao_statisticaCifar100(train_X, train_y, Val_X, Val_y, test_X, test_y, model, history)
plotAcuraciaLoss(history)

### [CIFAR100] Entropy Filter:
  * Filtered Dataset
  * Low Entropy in both training and validation data.


In [ ]:
(train_X, train_y), (test_X, test_y) = cifar100.load_data() # Carregar o conjunto de dados # Importar e Treinar
train_X, train_y = filtrar_entropia_classe(train_X, train_y,100)
train_X, Val_X, train_y, Val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y) # divisao do conjunto de dados
entHistogram(train_y, 1)
entHistogram(Val_y, 2)
entHistogram(test_y, 3)
print(len(train_X))
print(len(train_y))
print(len(test_X))
print(len(test_y))

In [ ]:
(train_X, train_y), (test_X, test_y) = cifar100.load_data() # Carregar o conjunto de dados # Importar e Treinar
tempoFiltroEntropiaInicio = time.time() #tempo para calculo do filtro
train_X, train_y = filtrar_entropia_classe(train_X, train_y,100)
tempoFiltroEntropiaFim = time.time() #Tempo final do calculo do filtro
train_X, Val_X, train_y, Val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y) # divisao do conjunto de dados
train_X, train_y, Val_X, Val_y, test_X, test_y = carregarPepararCifar100(train_X, train_y, Val_X, Val_y, test_X, test_y)
datagen = ImageDataGenerator(rotation_range=15,horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1)
datagen.fit(train_X)

model, learning_rate_reduction = criarModeloCifar100()

tempoTreinamento = time.time() #tempo para treinamento
history = model.fit(
    datagen.flow(train_X, train_y, batch_size=128),
    steps_per_epoch=len(train_X) / 128,
    epochs=50,
    validation_data=(Val_X, Val_y),
    callbacks=[learning_rate_reduction]
)
tempoFimTreino = time.time() #tempo para treinamento
tempoFinalTreino = round(tempoFimTreino - tempoTreinamento,2)
print("Tempo de treinamento(s): ", tempoFinalTreino)
TempoCalEntropia = round(tempoFiltroEntropiaFim - tempoFiltroEntropiaInicio,2)
print("Tempo de Calculo Entropia(s): ", TempoCalEntropia)
print("train_X: ",len(train_X))
print("train_y: ",len(train_y))
print("Val_X: ",len(Val_X))
print("Val_y: ",len(Val_y))
print("test_X: ",len(test_X))
print("test_y: ",len(test_y))
avaliacao_statisticaCifar100(train_X, train_y, Val_X, Val_y, test_X, test_y, model, history)
plotAcuraciaLoss(history)

### [CIFAR100] Entropy Filter:
  * Filtered Dataset
  * Low Entropy in both training and validation data.
  * Cross-validation


In [ ]:
def crossValidation(train_X,train_y,n_folds = 5):
    from sklearn.model_selection import KFold
    #divide seus dados em k folds, e treina no k−1 e valida em 1 fold, então a proporção de dados usados para validação é 1/k.
    #2 folds (50% de validação): Treina em 50% dos dados e valida nos outros 50%.
    #5 folds (20% de validação): Treina em 80% dos dados e valida em 20%.
    #10 folds (10% de validação): Treina em 90% dos dados e valida em 10%.
    n_fold = 5 if n_folds == 0 else n_folds
    datagen = ImageDataGenerator(rotation_range=15,horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1)
    datagen.fit(train_X)

    kf = KFold(n_splits = n_folds, shuffle = True, random_state = 42)  #divisão do conjunto de dados
    fold_no = 1
    for train_index, val_index in kf.split(train_X):
        train_data, val_data = train_X[train_index], train_X[val_index]
        train_label, val_label = train_y[train_index], train_y[val_index]

        print(f"Treinando a rede no folder numero: {fold_no}")
        model, learning_rate_reduction = criarModeloCifar100()
        datagen = ImageDataGenerator(rotation_range=15,horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1)
        datagen.fit(train_X)

        model, learning_rate_reduction = criarModeloCifar100()

        tempoTreinamento = time.time() #tempo para treinamento
        history = model.fit(
            datagen.flow(train_X, train_y, batch_size=128),
            steps_per_epoch=len(train_X) / 128,
            epochs=50,
            validation_data=(Val_X, Val_y),
            callbacks=[learning_rate_reduction]
        )
        tempoFimTreino = time.time() #tempo para treinamento
        tempoFinalTreino = round(tempoFimTreino - tempoTreinamento,2)
        print("Tempo de treinamento(s): ", tempoFinalTreino)
        TempoCalEntropia = round(tempoFiltroEntropiaFim - tempoFiltroEntropiaInicio,2)
        print("Tempo de Calculo Entropia(s): ", TempoCalEntropia)
        print("train_X: ",len(train_X))
        print("train_y: ",len(train_y))
        print("Val_X: ",len(Val_X))
        print("Val_y: ",len(Val_y))
        print("test_X: ",len(test_X))
        print("test_y: ",len(test_y))
        avaliacao_statisticaCifar100(train_X, train_y, Val_X, Val_y, test_X, test_y, model, history)
        plotAcuraciaLoss(history)

        fold_no += 1
        print("\n")



(train_X, train_y), (test_X, test_y) = cifar100.load_data() # Carregar o conjunto de dados # Importar e Treinar
tempoFiltro1 = time.time() #tempo de filtro
train_X, train_y, test_X, test_y = filtrar_entropia_median_train_teste(train_X, train_y, test_X, test_y,100) #novo filtro da reunião 03-10-2023
tempoFiltro2 = time.time() #tempo de filtro
train_X, train_y, test_X, test_y = carregarPepararValidacaoCruzadaCifar100(train_X, train_y, test_X, test_y) #padronização e categorização

train_X = np.concatenate([train_X, test_X], axis=0)
train_y = np.concatenate([train_y, test_y], axis=0)

#criar/instanciar o modelo para validação cruzada
crossValidation(train_X,train_y, 5)